# Day 5: Quantum ML Integration Project 🚀

## **Welcome to Day 5 - Quantum Machine Learning Integration!**

Today we'll bridge quantum chemistry and machine learning by working with the QM9 dataset and implementing state-of-the-art quantum ML models like SchNet. This is where quantum mechanics meets deep learning!

### **Project Overview:**
- **Section 1:** QM9 Dataset Mastery & Quantum Feature Engineering
- **Section 2:** SchNet Implementation & 3D Molecular Understanding
- **Section 3:** Delta Learning Framework for QM/ML Hybrid Models
- **Section 4:** Advanced Quantum ML Architectures
- **Section 5:** Production Pipeline & Integration Toolkit

### **Learning Objectives:**
- Master the QM9 dataset and quantum property prediction
- Implement SchNet for 3D molecular property prediction
- Build delta learning frameworks for QM/ML corrections
- Create advanced quantum ML architectures
- Develop production-ready quantum ML pipelines

### **Prerequisites from Previous Days:**
- Day 1: ML & Cheminformatics foundations
- Day 2: Deep learning for molecules
- Day 3: Molecular analysis pipelines
- Day 4: Quantum chemistry calculations

---

## **Section 1: QM9 Dataset Mastery & Quantum Feature Engineering** 🧬

Let's start by mastering the QM9 dataset - one of the most important quantum ML benchmarks!

In [ ]:
# Essential imports for Quantum ML
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional, Union, Any
import warnings
warnings.filterwarnings('ignore')

# Core scientific computing
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch_geometric.data import Data, Batch
from torch_geometric.nn import MessagePassing, global_add_pool, global_mean_pool
from torch_geometric.utils import add_self_loops, degree

# Chemistry and quantum computing
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, rdMolDescriptors
import deepchem as dc
from ase import Atoms
from ase.io import read, write

# ML and optimization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import optuna

# Visualization and analysis
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import joblib
import pickle
from pathlib import Path
import json
import time
from datetime import datetime
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("🎯 Quantum ML Integration Environment Ready!")
print(f"📊 PyTorch version: {torch.__version__}")
print(f"🧪 RDKit available: {Chem is not None}")
print(f"🔬 DeepChem version: {dc.__version__}")

In [ ]:
# 🎓 **DAY 5 ASSESSMENT FRAMEWORK INITIALIZATION**

print("🎓 DAY 5 ASSESSMENT FRAMEWORK INITIALIZATION")
print("="*70)

try:
    from assessment_framework import create_assessment, create_widget, create_dashboard
    print("✅ Assessment framework loaded successfully")
except ImportError:
    print("⚠️ Assessment framework not found. Please ensure assessment_framework.py is available.")
    print("📁 Expected location: same directory as this notebook")
    # Create a basic assessment object for fallback
    class BasicAssessment:
        def start_section(self, section): pass
        def end_section(self, section): pass
        def record_activity(self, activity, result, metadata=None): pass
        def get_progress_summary(self): return {"overall_score": 0.0, "section_scores": {}}
        def get_comprehensive_report(self): return {"activities": []}
        def save_final_report(self, filename): pass
    
    class BasicWidget:
        def display(self): print("📋 Assessment widget would appear here")
    
    def create_assessment(student_id, day=5, track="quantum_ml"):
        return BasicAssessment()
    
    def create_widget(assessment, section, concepts, activities):
        return BasicWidget()
    
    def create_dashboard(assessment):
        return BasicWidget()

# Student Information Collection
print("\n📝 Student Assessment Setup:")
student_id = input("Enter your student ID: ").strip()
if not student_id:
    student_id = f"student_day5_{np.random.randint(1000, 9999)}"
    print(f"Generated ID: {student_id}")

# Track Selection for Day 5 Quantum ML Specialization
print("\n🎯 Select your Quantum ML specialization track:")
print("1. 🧬 Quantum Molecular Property Prediction")
print("2. 🚀 Quantum Neural Network Development") 
print("3. 🔬 Quantum-Classical Hybrid Systems")
print("4. 🏭 Production Quantum ML Engineering")

track_choice = input("Enter choice (1-4): ").strip()
track_map = {
    "1": "quantum_molecular_prediction",
    "2": "quantum_neural_networks", 
    "3": "quantum_classical_hybrid",
    "4": "production_quantum_ml"
}

track_selected = track_map.get(track_choice, "quantum_molecular_prediction")
print(f"Selected track: {track_selected}")

# Initialize Assessment System
try:
    assessment = create_assessment(student_id=student_id, day=5, track=track_selected)
    print(f"✅ Assessment initialized for track: {track_selected}")
    print(f"👤 Student ID: {student_id}")
    
    # Start Day 5 assessment
    assessment.start_section("day_5_quantum_ml")
    print("\n🎯 Day 5 Assessment: Quantum ML Integration")
    print("📊 Progress tracking enabled - All activities will be recorded")
    
except Exception as e:
    print(f"⚠️ Assessment initialization warning: {e}")
    assessment = None

print("\n" + "="*70)
print("🚀 Ready to begin Day 5: Quantum ML Integration Project!")
print("="*70)

### **1.1 QM9 Dataset Handler - Professional Implementation**

In [ ]:
class QM9DatasetHandler:
    """
    Professional QM9 dataset handler with advanced preprocessing capabilities.
    
    The QM9 dataset contains ~134k small organic molecules with quantum chemical properties
    computed at the B3LYP/6-31G(2df,p) level of theory.
    """
    
    def __init__(self, cache_dir: str = "./qm9_cache"):
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)
        
        # QM9 property definitions with units and descriptions
        self.qm9_properties = {
            'mu': {'name': 'Dipole moment', 'unit': 'Debye', 'index': 0},
            'alpha': {'name': 'Polarizability', 'unit': 'Bohr^3', 'index': 1},
            'homo': {'name': 'HOMO energy', 'unit': 'Hartree', 'index': 2},
            'lumo': {'name': 'LUMO energy', 'unit': 'Hartree', 'index': 3},
            'gap': {'name': 'HOMO-LUMO gap', 'unit': 'Hartree', 'index': 4},
            'r2': {'name': 'Electronic spatial extent', 'unit': 'Bohr^2', 'index': 5},
            'zpve': {'name': 'Zero-point vibrational energy', 'unit': 'Hartree', 'index': 6},
            'u0': {'name': 'Internal energy at 0K', 'unit': 'Hartree', 'index': 7},
            'u298': {'name': 'Internal energy at 298K', 'unit': 'Hartree', 'index': 8},
            'h298': {'name': 'Enthalpy at 298K', 'unit': 'Hartree', 'index': 9},
            'g298': {'name': 'Free energy at 298K', 'unit': 'Hartree', 'index': 10},
            'cv': {'name': 'Heat capacity at 298K', 'unit': 'cal/(mol*K)', 'index': 11}
        }
        
        self.data = None
        self.molecular_graphs = []
        self.statistics = {}
        
    def load_qm9_dataset(self, subset_size: Optional[int] = None) -> pd.DataFrame:
        """
        Load and preprocess QM9 dataset with caching.
        """
        cache_file = self.cache_dir / f"qm9_processed_{subset_size or 'full'}.pkl"
        
        if cache_file.exists():
            logger.info(f"Loading cached QM9 data from {cache_file}")
            with open(cache_file, 'rb') as f:
                self.data = pickle.load(f)
            return self.data
        
        logger.info("Loading QM9 dataset from DeepChem...")
        try:
            # Load QM9 dataset using DeepChem
            qm9_loader = dc.molnet.load_qm9(featurizer='ECFP', split='random')
            train, valid, test = qm9_loader[0]
            
            # Combine all data
            all_smiles = np.concatenate([train[0], valid[0], test[0]])
            all_properties = np.concatenate([train[1], valid[1], test[1]])
            
            # Create DataFrame
            property_names = list(self.qm9_properties.keys())
            
            data_dict = {'smiles': all_smiles}
            for i, prop in enumerate(property_names):
                data_dict[prop] = all_properties[:, i]
            
            self.data = pd.DataFrame(data_dict)
            
            # Apply subset if requested
            if subset_size and subset_size < len(self.data):
                self.data = self.data.sample(n=subset_size, random_state=42).reset_index(drop=True)
            
            # Cache the processed data
            with open(cache_file, 'wb') as f:
                pickle.dump(self.data, f)
            
            logger.info(f"QM9 dataset loaded: {len(self.data)} molecules")
            return self.data
            
        except Exception as e:
            logger.error(f"Error loading QM9 dataset: {e}")
            # Fallback: create synthetic QM9-like data for demonstration
            return self._create_synthetic_qm9(subset_size or 1000)
    
    def _create_synthetic_qm9(self, n_samples: int = 1000) -> pd.DataFrame:
        """
        Create synthetic QM9-like data for demonstration purposes.
        """
        logger.warning("Creating synthetic QM9-like data for demonstration")
        
        # Generate simple organic molecules
        simple_smiles = [
            'C', 'CC', 'CCC', 'CCCC', 'CCCCC',  # Alkanes
            'C=C', 'CC=C', 'C=CC=C',  # Alkenes
            'C#C', 'CC#C',  # Alkynes
            'c1ccccc1', 'Cc1ccccc1',  # Aromatics
            'CO', 'CCO', 'CCCO',  # Alcohols
            'C=O', 'CC=O', 'CCC=O',  # Aldehydes/Ketones
            'CN', 'CCN', 'CCCN',  # Amines
        ]
        
        np.random.seed(42)
        smiles_list = np.random.choice(simple_smiles, n_samples)
        
        # Generate synthetic properties with realistic ranges
        data_dict = {'smiles': smiles_list}
        
        # Realistic property ranges based on QM9 statistics
        property_ranges = {
            'mu': (0, 5),  # Debye
            'alpha': (10, 100),  # Bohr^3
            'homo': (-0.3, -0.1),  # Hartree
            'lumo': (-0.1, 0.1),  # Hartree
            'gap': (0.05, 0.3),  # Hartree
            'r2': (20, 200),  # Bohr^2
            'zpve': (0.01, 0.3),  # Hartree
            'u0': (-500, -100),  # Hartree
            'u298': (-500, -100),  # Hartree
            'h298': (-500, -100),  # Hartree
            'g298': (-500, -100),  # Hartree
            'cv': (5, 50)  # cal/(mol*K)
        }
        
        for prop, (low, high) in property_ranges.items():
            data_dict[prop] = np.random.uniform(low, high, n_samples)
        
        self.data = pd.DataFrame(data_dict)
        return self.data
    
    def compute_statistics(self) -> Dict[str, Any]:
        """
        Compute comprehensive statistics for QM9 properties.
        """
        if self.data is None:
            raise ValueError("No data loaded. Call load_qm9_dataset first.")
        
        stats = {}
        
        for prop in self.qm9_properties.keys():
            if prop in self.data.columns:
                values = self.data[prop].values
                stats[prop] = {
                    'mean': np.mean(values),
                    'std': np.std(values),
                    'min': np.min(values),
                    'max': np.max(values),
                    'median': np.median(values),
                    'q25': np.percentile(values, 25),
                    'q75': np.percentile(values, 75),
                    'skewness': self._compute_skewness(values),
                    'kurtosis': self._compute_kurtosis(values)
                }
        
        self.statistics = stats
        return stats
    
    def _compute_skewness(self, values: np.ndarray) -> float:
        """Compute skewness of the distribution."""
        mean = np.mean(values)
        std = np.std(values)
        return np.mean(((values - mean) / std) ** 3)
    
    def _compute_kurtosis(self, values: np.ndarray) -> float:
        """Compute kurtosis of the distribution."""
        mean = np.mean(values)
        std = np.std(values)
        return np.mean(((values - mean) / std) ** 4) - 3
    
    def visualize_property_distributions(self, properties: Optional[List[str]] = None):
        """
        Create comprehensive visualization of QM9 property distributions.
        """
        if self.data is None:
            raise ValueError("No data loaded. Call load_qm9_dataset first.")
        
        if properties is None:
            properties = list(self.qm9_properties.keys())
        
        # Filter available properties
        available_props = [p for p in properties if p in self.data.columns]
        
        n_props = len(available_props)
        n_cols = 3
        n_rows = (n_props + n_cols - 1) // n_cols
        
        fig = make_subplots(
            rows=n_rows, cols=n_cols,
            subplot_titles=[f"{prop} ({self.qm9_properties[prop]['unit']})" 
                          for prop in available_props],
            vertical_spacing=0.1
        )
        
        for i, prop in enumerate(available_props):
            row = i // n_cols + 1
            col = i % n_cols + 1
            
            values = self.data[prop].values
            
            fig.add_trace(
                go.Histogram(
                    x=values,
                    name=prop,
                    nbinsx=50,
                    showlegend=False,
                    marker_color=px.colors.qualitative.Set3[i % len(px.colors.qualitative.Set3)]
                ),
                row=row, col=col
            )
        
        fig.update_layout(
            title="QM9 Property Distributions",
            height=300 * n_rows,
            showlegend=False
        )
        
        fig.show()
        
        return fig

# Initialize QM9 handler
qm9_handler = QM9DatasetHandler()
print("\n🎯 QM9 Dataset Handler initialized!")
print("📊 Ready to load and analyze quantum chemical properties")

### **1.2 Load and Explore QM9 Dataset**

In [ ]:
# Load QM9 dataset (using subset for faster processing)
print("Loading QM9 dataset...")
qm9_data = qm9_handler.load_qm9_dataset(subset_size=5000)  # Start with 5k molecules

print(f"\n📊 QM9 Dataset Overview:")
print(f"   • Total molecules: {len(qm9_data)}")
print(f"   • Properties: {len(qm9_handler.qm9_properties)}")
print(f"   • Data shape: {qm9_data.shape}")

# Display first few rows
print("\n🔍 Sample data:")
display(qm9_data.head())

# Compute and display statistics
print("\nComputing property statistics...")
stats = qm9_handler.compute_statistics()

# Create statistics summary table
stats_df = pd.DataFrame({
    prop: {
        'Mean': f"{data['mean']:.4f}",
        'Std': f"{data['std']:.4f}",
        'Min': f"{data['min']:.4f}",
        'Max': f"{data['max']:.4f}",
        'Unit': qm9_handler.qm9_properties[prop]['unit']
    }
    for prop, data in stats.items()
}).T

print("\n📈 QM9 Property Statistics:")
display(stats_df)

# Visualize property distributions
print("\nGenerating property distribution plots...")
fig = qm9_handler.visualize_property_distributions(['mu', 'alpha', 'homo', 'lumo', 'gap', 'cv'])

print("\n✅ QM9 dataset successfully loaded and analyzed!")

In [ ]:
# 🎯 **MID-SECTION EXERCISE CHECKPOINT 1.1: QM9 Data Mastery**

print("🎯 MID-SECTION EXERCISE CHECKPOINT 1.1: QM9 Data Mastery")
print("="*60)

# Quick hands-on exercise: Analyze specific molecular properties
exercise_widget_1_1 = create_widget(
    assessment, 
    section="1.1",
    concepts=[
        "QM9 dataset structure understanding",
        "Molecular property distributions",
        "Statistical analysis of quantum properties"
    ],
    activities=[
        "Loaded and explored QM9 dataset",
        "Analyzed property statistics and distributions", 
        "Interpreted quantum property ranges"
    ]
)

if assessment:
    assessment.record_activity(
        activity="qm9_data_exploration",
        result="completed",
        metadata={
            "dataset_size": len(qm9_data),
            "properties_analyzed": len(stats),
            "section": "1.1_mid_checkpoint"
        }
    )

exercise_widget_1_1.display()
print("🎓 Mid-section checkpoint 1.1 completed! Continue with feature engineering...")

### **1.3 Quantum Feature Engineering Framework**

In [ ]:
class QuantumFeatureEngineer:
    """
    Advanced quantum feature engineering for molecular property prediction.
    
    This class extracts and engineers features that are specifically relevant
    for quantum mechanical properties.
    """
    
    def __init__(self):
        self.feature_cache = {}
        self.scalers = {}
        
    def extract_molecular_features(self, smiles_list: List[str]) -> Dict[str, np.ndarray]:
        """
        Extract comprehensive molecular features for quantum property prediction.
        """
        features = {
            'constitutional': [],
            'topological': [],
            'electronic': [],
            'geometric': [],
            'quantum_descriptors': []
        }
        
        valid_molecules = []
        
        for smiles in smiles_list:
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                continue
                
            valid_molecules.append(smiles)
            
            # Constitutional descriptors
            const_features = self._extract_constitutional_features(mol)
            features['constitutional'].append(const_features)
            
            # Topological descriptors
            topo_features = self._extract_topological_features(mol)
            features['topological'].append(topo_features)
            
            # Electronic descriptors
            elec_features = self._extract_electronic_features(mol)
            features['electronic'].append(elec_features)
            
            # Geometric descriptors (if 3D coordinates available)
            geom_features = self._extract_geometric_features(mol)
            features['geometric'].append(geom_features)
            
            # Quantum-specific descriptors
            quantum_features = self._extract_quantum_descriptors(mol)
            features['quantum_descriptors'].append(quantum_features)
        
        # Convert to numpy arrays
        for key in features:
            if features[key]:
                features[key] = np.array(features[key])
            else:
                features[key] = np.array([]).reshape(0, 0)
        
        self.valid_molecules = valid_molecules
        return features
    
    def _extract_constitutional_features(self, mol: Chem.Mol) -> List[float]:
        """
        Extract constitutional molecular descriptors.
        """
        features = [
            mol.GetNumAtoms(),  # Number of atoms
            mol.GetNumBonds(),  # Number of bonds
            mol.GetNumHeavyAtoms(),  # Number of heavy atoms
            Descriptors.MolWt(mol),  # Molecular weight
            Descriptors.NumHeteroatoms(mol),  # Number of heteroatoms
            Descriptors.NumRotatableBonds(mol),  # Number of rotatable bonds
            Descriptors.NumAromaticRings(mol),  # Number of aromatic rings
            Descriptors.NumSaturatedRings(mol),  # Number of saturated rings
            Descriptors.RingCount(mol),  # Total ring count
            Descriptors.FractionCsp3(mol),  # Fraction of sp3 carbons
        ]
        
        # Atom type counts
        atom_counts = {'C': 0, 'N': 0, 'O': 0, 'F': 0, 'P': 0, 'S': 0, 'Cl': 0, 'Br': 0}
        for atom in mol.GetAtoms():
            symbol = atom.GetSymbol()
            if symbol in atom_counts:
                atom_counts[symbol] += 1
        
        features.extend(list(atom_counts.values()))
        return features
    
    def _extract_topological_features(self, mol: Chem.Mol) -> List[float]:
        """
        Extract topological molecular descriptors.
        """
        features = [
            Descriptors.Chi0(mol),  # Chi0 connectivity index
            Descriptors.Chi1(mol),  # Chi1 connectivity index
            Descriptors.BalabanJ(mol),  # Balaban J index
            Descriptors.Kappa1(mol),  # Kappa1 shape index
            Descriptors.Kappa2(mol),  # Kappa2 shape index
            Descriptors.Kappa3(mol),  # Kappa3 shape index
            rdMolDescriptors.CalcNumSpiroAtoms(mol),  # Number of spiro atoms
            rdMolDescriptors.CalcNumBridgeheadAtoms(mol),  # Number of bridgehead atoms
        ]
        
        # Handle potential None values
        features = [f if f is not None else 0.0 for f in features]
        return features
    
    def _extract_electronic_features(self, mol: Chem.Mol) -> List[float]:
        """
        Extract electronic molecular descriptors.
        """
        features = [
            Descriptors.NumValenceElectrons(mol),  # Number of valence electrons
            Descriptors.MaxPartialCharge(mol),  # Maximum partial charge
            Descriptors.MinPartialCharge(mol),  # Minimum partial charge
            Descriptors.MaxAbsPartialCharge(mol),  # Maximum absolute partial charge
            Descriptors.MinAbsPartialCharge(mol),  # Minimum absolute partial charge
        ]
        
        # Compute partial charges using Gasteiger method
        try:
            AllChem.ComputeGasteigerCharges(mol)
            charges = [float(atom.GetProp('_GasteigerCharge')) for atom in mol.GetAtoms()]
            if charges:
                features.extend([
                    np.mean(charges),  # Mean partial charge
                    np.std(charges),   # Std of partial charges
                    np.sum(np.abs(charges)),  # Sum of absolute charges
                ])
            else:
                features.extend([0.0, 0.0, 0.0])
        except:
            features.extend([0.0, 0.0, 0.0])
        
        # Handle potential None values
        features = [f if f is not None else 0.0 for f in features]
        return features
    
    def _extract_geometric_features(self, mol: Chem.Mol) -> List[float]:
        """
        Extract geometric molecular descriptors.
        """
        # Generate 3D coordinates if not present
        mol_copy = Chem.AddHs(mol)
        
        try:
            AllChem.EmbedMolecule(mol_copy, randomSeed=42)
            AllChem.OptimizeMoleculeConfs(mol_copy)
            
            features = [
                Descriptors.Asphericity(mol_copy),  # Asphericity
                Descriptors.Eccentricity(mol_copy),  # Eccentricity
                Descriptors.InertialShapeFactor(mol_copy),  # Inertial shape factor
                Descriptors.RadiusOfGyration(mol_copy),  # Radius of gyration
                Descriptors.SpherocityIndex(mol_copy),  # Spherocity index
            ]
        except:
            # If 3D generation fails, use default values
            features = [0.0, 0.0, 0.0, 0.0, 0.0]
        
        # Handle potential None values
        features = [f if f is not None else 0.0 for f in features]
        return features
    
    def _extract_quantum_descriptors(self, mol: Chem.Mol) -> List[float]:
        """
        Extract quantum-chemistry specific descriptors.
        """
        features = []
        
        # Aromaticity and conjugation features
        aromatic_atoms = sum(1 for atom in mol.GetAtoms() if atom.GetIsAromatic())
        aromatic_bonds = sum(1 for bond in mol.GetBonds() if bond.GetIsAromatic())
        conjugated_bonds = sum(1 for bond in mol.GetBonds() if bond.GetIsConjugated())
        
        features.extend([
            aromatic_atoms / mol.GetNumAtoms() if mol.GetNumAtoms() > 0 else 0,
            aromatic_bonds / mol.GetNumBonds() if mol.GetNumBonds() > 0 else 0,
            conjugated_bonds / mol.GetNumBonds() if mol.GetNumBonds() > 0 else 0,
        ])
        
        # Hybridization state features
        hybridization_counts = {'SP': 0, 'SP2': 0, 'SP3': 0}
        for atom in mol.GetAtoms():
            hyb = str(atom.GetHybridization())
            if hyb in hybridization_counts:
                hybridization_counts[hyb] += 1
        
        total_atoms = mol.GetNumAtoms()
        if total_atoms > 0:
            features.extend([
                hybridization_counts['SP'] / total_atoms,
                hybridization_counts['SP2'] / total_atoms,
                hybridization_counts['SP3'] / total_atoms,
            ])
        else:
            features.extend([0.0, 0.0, 0.0])
        
        # Formal charge distribution
        formal_charges = [atom.GetFormalCharge() for atom in mol.GetAtoms()]
        if formal_charges:
            features.extend([
                np.sum(formal_charges),  # Total formal charge
                np.sum(np.abs(formal_charges)),  # Sum of absolute formal charges
                len([c for c in formal_charges if c != 0]),  # Number of charged atoms
            ])
        else:
            features.extend([0.0, 0.0, 0.0])
        
        return features
    
    def create_feature_matrix(self, features_dict: Dict[str, np.ndarray]) -> Tuple[np.ndarray, List[str]]:
        """
        Combine all feature types into a single matrix.
        """
        feature_arrays = []
        feature_names = []
        
        for feature_type, features in features_dict.items():
            if features.size > 0:
                feature_arrays.append(features)
                
                # Generate feature names
                n_features = features.shape[1]
                names = [f"{feature_type}_{i}" for i in range(n_features)]
                feature_names.extend(names)
        
        if feature_arrays:
            combined_features = np.hstack(feature_arrays)
        else:
            combined_features = np.array([]).reshape(0, 0)
        
        return combined_features, feature_names
    
    def scale_features(self, features: np.ndarray, method: str = 'standard') -> np.ndarray:
        """
        Scale features using specified method.
        """
        if method not in self.scalers:
            if method == 'standard':
                self.scalers[method] = StandardScaler()
            elif method == 'robust':
                self.scalers[method] = RobustScaler()
            else:
                raise ValueError(f"Unknown scaling method: {method}")
        
        scaler = self.scalers[method]
        
        if not hasattr(scaler, 'mean_'):
            # Fit the scaler
            scaled_features = scaler.fit_transform(features)
        else:
            # Transform using existing fit
            scaled_features = scaler.transform(features)
        
        return scaled_features

# Initialize feature engineer
feature_engineer = QuantumFeatureEngineer()
print("\n🎯 Quantum Feature Engineer initialized!")
print("🔬 Ready to extract quantum-specific molecular features")

### **1.4 Extract and Analyze Molecular Features**

In [ ]:
# Extract comprehensive molecular features
print("Extracting molecular features from QM9 dataset...")
smiles_list = qm9_data['smiles'].tolist()

# Extract features (this may take a few minutes)
start_time = time.time()
features_dict = feature_engineer.extract_molecular_features(smiles_list)
feature_extraction_time = time.time() - start_time

print(f"\n⏱️ Feature extraction completed in {feature_extraction_time:.2f} seconds")

# Display feature statistics
for feature_type, features in features_dict.items():
    if features.size > 0:
        print(f"   • {feature_type}: {features.shape[1]} features, {features.shape[0]} molecules")
    else:
        print(f"   • {feature_type}: No features extracted")

# Create combined feature matrix
print("\nCombining features into matrix...")
feature_matrix, feature_names = feature_engineer.create_feature_matrix(features_dict)

print(f"\n📊 Combined Feature Matrix:")
print(f"   • Shape: {feature_matrix.shape}")
print(f"   • Total features: {len(feature_names)}")
print(f"   • Valid molecules: {len(feature_engineer.valid_molecules)}")

# Scale features
print("\nScaling features...")
scaled_features = feature_engineer.scale_features(feature_matrix, method='standard')

print(f"✅ Features extracted and scaled successfully!")
print(f"   • Original range: [{feature_matrix.min():.3f}, {feature_matrix.max():.3f}]")
print(f"   • Scaled range: [{scaled_features.min():.3f}, {scaled_features.max():.3f}]")

### **1.5 Feature-Property Correlation Analysis**

In [ ]:
# Align QM9 data with valid molecules
valid_indices = [i for i, smiles in enumerate(qm9_data['smiles']) 
                if smiles in feature_engineer.valid_molecules]
aligned_qm9_data = qm9_data.iloc[valid_indices].reset_index(drop=True)

print(f"Aligned dataset: {len(aligned_qm9_data)} molecules")

# Compute correlations between features and properties
property_columns = ['mu', 'alpha', 'homo', 'lumo', 'gap', 'cv']
available_properties = [p for p in property_columns if p in aligned_qm9_data.columns]

correlations = {}
for prop in available_properties:
    property_values = aligned_qm9_data[prop].values
    
    # Compute correlations with each feature type
    correlations[prop] = {}
    
    start_idx = 0
    for feature_type, features in features_dict.items():
        if features.size > 0:
            end_idx = start_idx + features.shape[1]
            
            # Compute correlation for this feature type
            feature_subset = scaled_features[:, start_idx:end_idx]
            corr_values = []
            
            for i in range(feature_subset.shape[1]):
                corr = np.corrcoef(property_values, feature_subset[:, i])[0, 1]
                if not np.isnan(corr):
                    corr_values.append(abs(corr))
                else:
                    corr_values.append(0.0)
            
            correlations[prop][feature_type] = {
                'max': max(corr_values) if corr_values else 0.0,
                'mean': np.mean(corr_values) if corr_values else 0.0,
                'top_features': sorted(enumerate(corr_values), key=lambda x: x[1], reverse=True)[:3]
            }
            
            start_idx = end_idx

# Display correlation results
print("\n🔍 Feature-Property Correlation Analysis:")
print("=" * 60)

for prop in available_properties:
    prop_info = qm9_handler.qm9_properties[prop]
    print(f"\n📊 {prop_info['name']} ({prop_info['unit']}):")
    
    for feature_type, corr_data in correlations[prop].items():
        print(f"   • {feature_type.title()}: max={corr_data['max']:.3f}, mean={corr_data['mean']:.3f}")

# Create correlation heatmap for top features
def create_correlation_heatmap(property_name: str, top_n: int = 20):
    """Create correlation heatmap for top features."""
    if property_name not in available_properties:
        return None
    
    property_values = aligned_qm9_data[property_name].values
    
    # Find top correlating features across all types
    all_correlations = []
    feature_labels = []
    
    start_idx = 0
    for feature_type, features in features_dict.items():
        if features.size > 0:
            end_idx = start_idx + features.shape[1]
            feature_subset = scaled_features[:, start_idx:end_idx]
            
            for i in range(feature_subset.shape[1]):
                corr = np.corrcoef(property_values, feature_subset[:, i])[0, 1]
                if not np.isnan(corr):
                    all_correlations.append(abs(corr))
                    feature_labels.append(f"{feature_type}_{i}")
                else:
                    all_correlations.append(0.0)
                    feature_labels.append(f"{feature_type}_{i}")
            
            start_idx = end_idx
    
    # Get top correlating features
    top_indices = np.argsort(all_correlations)[-top_n:][::-1]
    top_features = scaled_features[:, top_indices]
    top_labels = [feature_labels[i] for i in top_indices]
    
    # Create correlation matrix
    corr_matrix = np.corrcoef(np.column_stack([property_values, top_features.T]))
    
    # Create heatmap
    fig = go.Figure(data=go.Heatmap(
        z=corr_matrix[1:, 1:],  # Exclude property vs property correlation
        x=top_labels,
        y=top_labels,
        colorscale='RdBu',
        zmid=0,
        text=np.round(corr_matrix[1:, 1:], 3),
        texttemplate="%{text}",
        textfont={"size": 8},
        showscale=True
    ))
    
    fig.update_layout(
        title=f"Top {top_n} Feature Correlations for {property_name}",
        xaxis_title="Features",
        yaxis_title="Features",
        height=600,
        width=800
    )
    
    return fig

# Show correlation heatmap for HOMO energy
print(f"\nCreating correlation heatmap for HOMO energy...")
if 'homo' in available_properties:
    homo_heatmap = create_correlation_heatmap('homo', top_n=15)
    if homo_heatmap:
        homo_heatmap.show()

print("✅ Feature-property correlation analysis completed!")

### **1.6 Baseline ML Models for Quantum Property Prediction**

In [ ]:
class QuantumPropertyPredictor:
    """
    Baseline ML models for quantum property prediction.
    """
    
    def __init__(self):
        self.models = {}
        self.scalers = {}
        self.results = {}
        
    def train_baseline_models(self, X: np.ndarray, y_dict: Dict[str, np.ndarray], 
                            test_size: float = 0.2, random_state: int = 42):
        """
        Train baseline models for each quantum property.
        """
        self.results = {}
        
        for property_name, y_values in y_dict.items():
            print(f"\n🎯 Training models for {property_name}...")
            
            # Split data
            X_train, X_test, y_train, y_test = train_test_split(
                X, y_values, test_size=test_size, random_state=random_state
            )
            
            # Train Random Forest
            rf_model = RandomForestRegressor(
                n_estimators=100,
                max_depth=15,
                min_samples_split=5,
                min_samples_leaf=2,
                random_state=random_state,
                n_jobs=-1
            )
            
            rf_model.fit(X_train, y_train)
            rf_pred = rf_model.predict(X_test)
            
            # Compute metrics
            rf_mae = mean_absolute_error(y_test, rf_pred)
            rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
            rf_r2 = r2_score(y_test, rf_pred)
            
            # Store results
            self.models[property_name] = rf_model
            self.results[property_name] = {
                'mae': rf_mae,
                'rmse': rf_rmse,
                'r2': rf_r2,
                'y_test': y_test,
                'y_pred': rf_pred
            }
            
            print(f"   • MAE: {rf_mae:.4f}")
            print(f"   • RMSE: {rf_rmse:.4f}")
            print(f"   • R²: {rf_r2:.4f}")
    
    def create_prediction_plots(self):
        """
        Create prediction vs actual plots for all properties.
        """
        n_props = len(self.results)
        n_cols = 3
        n_rows = (n_props + n_cols - 1) // n_cols
        
        fig = make_subplots(
            rows=n_rows, cols=n_cols,
            subplot_titles=list(self.results.keys()),
            vertical_spacing=0.1
        )
        
        for i, (prop, results) in enumerate(self.results.items()):
            row = i // n_cols + 1
            col = i % n_cols + 1
            
            y_test = results['y_test']
            y_pred = results['y_pred']
            
            # Scatter plot
            fig.add_trace(
                go.Scatter(
                    x=y_test,
                    y=y_pred,
                    mode='markers',
                    name=f'{prop}',
                    showlegend=False,
                    marker=dict(
                        size=4,
                        opacity=0.6,
                        color=px.colors.qualitative.Set3[i % len(px.colors.qualitative.Set3)]
                    )
                ),
                row=row, col=col
            )
            
            # Perfect prediction line
            min_val = min(y_test.min(), y_pred.min())
            max_val = max(y_test.max(), y_pred.max())
            
            fig.add_trace(
                go.Scatter(
                    x=[min_val, max_val],
                    y=[min_val, max_val],
                    mode='lines',
                    line=dict(dash='dash', color='red'),
                    showlegend=False
                ),
                row=row, col=col
            )
            
            # Update subplot labels
            fig.update_xaxes(title_text="Actual", row=row, col=col)
            fig.update_yaxes(title_text="Predicted", row=row, col=col)
        
        fig.update_layout(
            title="Baseline Model Performance: Predicted vs Actual",
            height=300 * n_rows,
            showlegend=False
        )
        
        return fig

# Initialize predictor and train baseline models
predictor = QuantumPropertyPredictor()

# Prepare target variables
target_dict = {}
for prop in available_properties:
    target_dict[prop] = aligned_qm9_data[prop].values

print("🎯 Training baseline Random Forest models...")
predictor.train_baseline_models(scaled_features, target_dict)

# Create performance plots
print("\nCreating prediction performance plots...")
performance_fig = predictor.create_prediction_plots()
performance_fig.show()

# Summary of results
print("\n📊 Baseline Model Performance Summary:")
print("=" * 50)
for prop, results in predictor.results.items():
    prop_info = qm9_handler.qm9_properties[prop]
    print(f"{prop_info['name']:25} | MAE: {results['mae']:.4f} | R²: {results['r2']:.3f}")

print("\n✅ Section 1 Complete: QM9 Dataset Mastery & Quantum Feature Engineering")
print("🎯 Ready to move to Section 2: SchNet Implementation & 3D Molecular Understanding")

In [ ]:
# 📋 SECTION 1 CHECKPOINT ASSESSMENT: QM9 Dataset Mastery & Quantum Feature Engineering
print("\n" + "="*80)
print("📋 SECTION 1 CHECKPOINT ASSESSMENT: QM9 Dataset Mastery & Quantum Feature Engineering")
print("="*80)

if assessment:
    # Record section completion
    assessment.record_activity(
        "section_1_completion", 
        "completed",
        {"section": "QM9 Dataset Mastery & Quantum Feature Engineering", "timestamp": datetime.now().isoformat()}
    )

# Create assessment widget for Section 1
section1_widget = create_widget(
    assessment=assessment,
    section="Section 1: QM9 Dataset Mastery & Quantum Feature Engineering",
    concepts=[
        "QM9 dataset structure and properties understanding",
        "Quantum chemical property analysis and visualization", 
        "3D molecular coordinate handling and preprocessing",
        "Molecular graph construction from 3D structures",
        "Quantum feature engineering and scaling techniques",
        "Data splits and cross-validation for quantum ML",
        "Performance metrics for quantum property prediction"
    ],
    activities=[
        "Successfully loaded and explored QM9 dataset",
        "Analyzed quantum chemical properties (HOMO, LUMO, dipole moment)",
        "Implemented molecular graph construction pipeline",
        "Created 3D visualization of molecular structures",
        "Generated quantum-aware molecular features",
        "Established training/validation splits for QM9 data",
        "Computed baseline statistics and property distributions"
    ]
)

# Display the interactive assessment
section1_widget.display()

# Progress tracking
if assessment:
    print(f"\n📊 Current Progress: {assessment.get_progress_summary()['overall_score']:.1f}%")
    print("🎯 Section 1 assessment completed - proceed when ready!")

---

## **Section 2: SchNet Implementation & 3D Molecular Understanding** 🌐

Now let's implement SchNet, a state-of-the-art deep learning architecture for 3D molecular property prediction that leverages continuous-filter convolutional layers.

In [ ]:
# Additional imports for SchNet implementation
import torch_geometric
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import MessagePassing, global_add_pool, global_mean_pool
from torch_geometric.utils import add_self_loops, remove_self_loops, softmax
from torch_scatter import scatter_add, scatter_mean
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

print("🚀 SchNet Implementation Environment Ready!")
print(f"   • PyTorch Geometric version: {torch_geometric.__version__}")

### **2.1 SchNet Architecture Implementation**

In [ ]:
class CFConv(MessagePassing):
    """
    Continuous-filter convolutional layer as used in SchNet.
    
    This layer uses continuous filters to model interactions between atoms
    based on their distances, making it suitable for 3D molecular modeling.
    """
    
    def __init__(self, in_channels: int, out_channels: int, num_filters: int, 
                 cutoff: float = 10.0, smooth: bool = True):
        super(CFConv, self).__init__(aggr='add')
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_filters = num_filters
        self.cutoff = cutoff
        self.smooth = smooth
        
        # Filter-generating network
        self.filter_network = nn.Sequential(
            nn.Linear(1, num_filters),
            nn.Softplus(),
            nn.Linear(num_filters, num_filters),
            nn.Softplus(),
            nn.Linear(num_filters, in_channels * out_channels)
        )
        
        # Gate network for smooth cutoff
        if smooth:
            self.gate_network = nn.Sequential(
                nn.Linear(1, num_filters),
                nn.Softplus(),
                nn.Linear(num_filters, 1),
                nn.Sigmoid()
            )
        
        self.reset_parameters()
    
    def reset_parameters(self):
        """Initialize parameters."""
        for layer in self.filter_network:
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()
        
        if self.smooth:
            for layer in self.gate_network:
                if hasattr(layer, 'reset_parameters'):
                    layer.reset_parameters()
    
    def forward(self, x: torch.Tensor, edge_index: torch.Tensor, 
                edge_attr: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of continuous-filter convolution.
        
        Args:
            x: Node features [num_nodes, in_channels]
            edge_index: Graph connectivity [2, num_edges]
            edge_attr: Edge attributes (distances) [num_edges, 1]
        """
        # Generate continuous filters based on distances
        edge_filters = self.filter_network(edge_attr)
        edge_filters = edge_filters.view(-1, self.in_channels, self.out_channels)
        
        # Apply smooth cutoff if enabled
        if self.smooth:
            cutoff_values = self.gate_network(edge_attr)
            edge_filters = edge_filters * cutoff_values.unsqueeze(-1)
        
        # Propagate messages
        return self.propagate(edge_index, x=x, edge_filters=edge_filters)
    
    def message(self, x_j: torch.Tensor, edge_filters: torch.Tensor) -> torch.Tensor:
        """
        Message function: apply continuous filters to neighboring node features.
        """
        # x_j: [num_edges, in_channels]
        # edge_filters: [num_edges, in_channels, out_channels]
        
        # Apply filters via batch matrix multiplication
        messages = torch.bmm(x_j.unsqueeze(1), edge_filters).squeeze(1)
        return messages


class InteractionBlock(nn.Module):
    """
    SchNet interaction block combining CFConv with residual connections.
    """
    
    def __init__(self, hidden_channels: int, num_filters: int, cutoff: float = 10.0):
        super(InteractionBlock, self).__init__()
        
        self.cfconv = CFConv(hidden_channels, hidden_channels, num_filters, cutoff)
        
        # Dense layers for feature transformation
        self.dense1 = nn.Linear(hidden_channels, hidden_channels)
        self.dense2 = nn.Linear(hidden_channels, hidden_channels)
        
        # Activation functions
        self.activation = nn.Softplus()
        
    def forward(self, x: torch.Tensor, edge_index: torch.Tensor, 
                edge_attr: torch.Tensor) -> torch.Tensor:
        """Forward pass with residual connection."""
        # Store input for residual connection
        residual = x
        
        # Apply continuous-filter convolution
        x = self.cfconv(x, edge_index, edge_attr)
        
        # Apply dense transformations
        x = self.dense1(x)
        x = self.activation(x)
        x = self.dense2(x)
        
        # Add residual connection
        return x + residual


class SchNet(nn.Module):
    """
    SchNet architecture for molecular property prediction.
    
    SchNet uses continuous-filter convolutional layers to learn representations
    of molecules that respect rotational and translational invariance.
    """
    
    def __init__(self, hidden_channels: int = 128, num_filters: int = 128,
                 num_interactions: int = 6, num_gaussians: int = 50,
                 cutoff: float = 10.0, readout: str = 'add'):
        super(SchNet, self).__init__()
        
        self.hidden_channels = hidden_channels
        self.num_filters = num_filters
        self.num_interactions = num_interactions
        self.num_gaussians = num_gaussians
        self.cutoff = cutoff
        self.readout = readout
        
        # Atomic number embedding (assuming max atomic number 100)
        self.atomic_embedding = nn.Embedding(100, hidden_channels)
        
        # Distance expansion using Gaussian basis functions
        self.distance_expansion = GaussianSmearing(0.0, cutoff, num_gaussians)
        
        # Interaction blocks
        self.interactions = nn.ModuleList([
            InteractionBlock(hidden_channels, num_filters, cutoff)
            for _ in range(num_interactions)
        ])
        
        # Output network
        self.output_network = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels // 2),
            nn.Softplus(),
            nn.Linear(hidden_channels // 2, 1)
        )
        
        self.reset_parameters()
    
    def reset_parameters(self):
        """Initialize all parameters."""
        self.atomic_embedding.reset_parameters()
        
        for interaction in self.interactions:
            interaction.dense1.reset_parameters()
            interaction.dense2.reset_parameters()
            interaction.cfconv.reset_parameters()
        
        for layer in self.output_network:
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()
    
    def forward(self, batch: Data) -> torch.Tensor:
        """
        Forward pass of SchNet.
        
        Args:
            batch: PyTorch Geometric batch containing:
                - x: Atomic numbers [num_atoms]
                - pos: 3D coordinates [num_atoms, 3]
                - edge_index: Graph connectivity [2, num_edges]
                - batch: Batch assignment [num_atoms]
        """
        x, pos, edge_index, batch_idx = batch.x, batch.pos, batch.edge_index, batch.batch
        
        # Compute edge distances
        row, col = edge_index
        edge_distances = torch.norm(pos[row] - pos[col], dim=1).unsqueeze(-1)
        
        # Filter edges by cutoff distance
        edge_mask = edge_distances.squeeze() <= self.cutoff
        edge_index = edge_index[:, edge_mask]
        edge_distances = edge_distances[edge_mask]
        
        # Expand distances using Gaussian basis
        edge_attr = self.distance_expansion(edge_distances)
        
        # Embed atomic numbers
        x = self.atomic_embedding(x.long())
        
        # Apply interaction blocks
        for interaction in self.interactions:
            x = interaction(x, edge_index, edge_attr)
        
        # Global pooling
        if self.readout == 'add':
            x = global_add_pool(x, batch_idx)
        elif self.readout == 'mean':
            x = global_mean_pool(x, batch_idx)
        
        # Final output
        return self.output_network(x)


class GaussianSmearing(nn.Module):
    """
    Gaussian smearing of distances for continuous representations.
    """
    
    def __init__(self, start: float = 0.0, stop: float = 5.0, num_gaussians: int = 50):
        super(GaussianSmearing, self).__init__()
        
        offset = torch.linspace(start, stop, num_gaussians)
        self.coeff = -0.5 / (offset[1] - offset[0]).item() ** 2
        self.register_buffer('offset', offset)
    
    def forward(self, dist: torch.Tensor) -> torch.Tensor:
        """Apply Gaussian smearing to distances."""
        dist = dist.view(-1, 1) - self.offset.view(1, -1)
        return torch.exp(self.coeff * torch.pow(dist, 2))

print("✅ SchNet architecture implemented!")
print("🎯 Ready for 3D molecular property prediction")

### **2.2 3D Molecular Graph Construction**

In [ ]:
class MolecularGraphBuilder:
    """
    Build 3D molecular graphs from SMILES strings for SchNet.
    """
    
    def __init__(self, cutoff: float = 10.0):
        self.cutoff = cutoff
        
    def smiles_to_graph(self, smiles: str, target: Optional[float] = None) -> Optional[Data]:
        """
        Convert SMILES to 3D molecular graph.
        
        Args:
            smiles: SMILES string
            target: Optional target property value
            
        Returns:
            PyTorch Geometric Data object or None if conversion fails
        """
        try:
            # Create molecule object
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                return None
            
            # Add hydrogens and generate 3D coordinates
            mol = Chem.AddHs(mol)
            
            # Generate 3D coordinates using ETKDG
            params = AllChem.ETKDGv3()
            params.randomSeed = 42
            
            if AllChem.EmbedMolecule(mol, params) == -1:
                return None
            
            # Optimize molecular geometry
            AllChem.OptimizeMoleculeConfs(mol, maxIters=1000)
            
            # Extract atomic information
            atomic_numbers = []
            positions = []
            
            conf = mol.GetConformer()
            for atom in mol.GetAtoms():
                atomic_numbers.append(atom.GetAtomicNum())
                pos = conf.GetAtomPosition(atom.GetIdx())
                positions.append([pos.x, pos.y, pos.z])
            
            # Convert to tensors
            x = torch.tensor(atomic_numbers, dtype=torch.long)
            pos = torch.tensor(positions, dtype=torch.float)
            
            # Build edge index based on distance cutoff
            edge_index = self._build_edge_index(pos, self.cutoff)
            
            # Create Data object
            data = Data(x=x, pos=pos, edge_index=edge_index)
            
            if target is not None:
                data.y = torch.tensor([target], dtype=torch.float)
            
            return data
            
        except Exception as e:
            logger.warning(f"Failed to convert SMILES {smiles}: {e}")
            return None
    
    def _build_edge_index(self, pos: torch.Tensor, cutoff: float) -> torch.Tensor:
        """
        Build edge index based on distance cutoff.
        """
        num_atoms = pos.size(0)
        
        # Compute pairwise distances
        distances = torch.cdist(pos, pos)
        
        # Create adjacency matrix based on cutoff
        adj_matrix = (distances <= cutoff) & (distances > 0)  # Exclude self-loops
        
        # Convert to edge index format
        edge_index = adj_matrix.nonzero().t().contiguous()
        
        return edge_index
    
    def create_dataset(self, smiles_list: List[str], targets: Optional[List[float]] = None) -> List[Data]:
        """
        Create a dataset of molecular graphs.
        """
        dataset = []
        
        for i, smiles in enumerate(smiles_list):
            target = targets[i] if targets is not None else None
            
            graph = self.smiles_to_graph(smiles, target)
            if graph is not None:
                dataset.append(graph)
        
        logger.info(f"Successfully created {len(dataset)} molecular graphs from {len(smiles_list)} SMILES")
        return dataset

# Initialize graph builder
graph_builder = MolecularGraphBuilder(cutoff=8.0)
print("🎯 Molecular Graph Builder initialized!")
print("🔬 Ready to convert SMILES to 3D molecular graphs")

### **2.3 Create 3D Molecular Graphs for Training**

In [ ]:
# Create 3D molecular graphs from QM9 dataset
print("Converting QM9 molecules to 3D graphs...")

# Use a subset for faster processing
subset_size = 1000
subset_data = aligned_qm9_data.head(subset_size)

# Create molecular graphs for HOMO energy prediction
homo_values = subset_data['homo'].values
smiles_subset = subset_data['smiles'].tolist()

print(f"Building 3D graphs for {len(smiles_subset)} molecules...")
start_time = time.time()

molecular_graphs = graph_builder.create_dataset(smiles_subset, homo_values.tolist())

build_time = time.time() - start_time
print(f"⏱️ Graph construction completed in {build_time:.2f} seconds")

print(f"\n📊 3D Molecular Graph Dataset:")
print(f"   • Total graphs: {len(molecular_graphs)}")
print(f"   • Success rate: {len(molecular_graphs)/len(smiles_subset)*100:.1f}%")

# Analyze graph properties
if molecular_graphs:
    sample_graph = molecular_graphs[0]
    print(f"\n🔍 Sample Graph Properties:")
    print(f"   • Number of atoms: {sample_graph.x.size(0)}")
    print(f"   • Number of edges: {sample_graph.edge_index.size(1)}")
    print(f"   • Node features shape: {sample_graph.x.shape}")
    print(f"   • Position features shape: {sample_graph.pos.shape}")
    print(f"   • Target value: {sample_graph.y.item():.4f}")

# Analyze dataset statistics
num_atoms = [graph.x.size(0) for graph in molecular_graphs]
num_edges = [graph.edge_index.size(1) for graph in molecular_graphs]

print(f"\n📈 Dataset Statistics:")
print(f"   • Atoms per molecule: {np.mean(num_atoms):.1f} ± {np.std(num_atoms):.1f}")
print(f"   • Edges per molecule: {np.mean(num_edges):.1f} ± {np.std(num_edges):.1f}")
print(f"   • Min/Max atoms: {np.min(num_atoms)}/{np.max(num_atoms)}")
print(f"   • Min/Max edges: {np.min(num_edges)}/{np.max(num_edges)}")

print("✅ 3D molecular graphs successfully created!")

In [ ]:
# 🎯 **MID-SECTION EXERCISE CHECKPOINT 2.1: 3D Molecular Graph Construction**

print("🎯 MID-SECTION EXERCISE CHECKPOINT 2.1: 3D Molecular Graph Construction")
print("="*70)

# Quick hands-on exercise: Analyze 3D molecular graphs
exercise_widget_2_1 = create_widget(
    assessment, 
    section="2.1",
    concepts=[
        "3D molecular graph representation",
        "Node and edge feature engineering",
        "Geometric deep learning principles"
    ],
    activities=[
        "Built 3D molecular graphs from SMILES",
        "Analyzed graph statistics and properties", 
        "Prepared data for SchNet training"
    ]
)

if assessment:
    assessment.record_activity(
        activity="3d_graph_construction",
        result="completed",
        metadata={
            "graphs_created": len(molecular_graphs),
            "success_rate": len(molecular_graphs)/len(smiles_subset)*100,
            "section": "2.1_mid_checkpoint"
        }
    )

exercise_widget_2_1.display()
print("🎓 Mid-section checkpoint 2.1 completed! Continue with SchNet training...")

### **2.4 SchNet Training Pipeline**

In [ ]:
class SchNetTrainer:
    """
    Training pipeline for SchNet molecular property prediction.
    """
    
    def __init__(self, model: SchNet, device: str = 'cpu'):
        self.model = model.to(device)
        self.device = device
        self.training_history = {
            'train_loss': [],
            'val_loss': [],
            'train_mae': [],
            'val_mae': []
        }
        
    def prepare_data(self, dataset: List[Data], train_ratio: float = 0.8, 
                    batch_size: int = 32) -> Tuple[DataLoader, DataLoader]:
        """
        Prepare training and validation data loaders.
        """
        # Split dataset
        train_size = int(len(dataset) * train_ratio)
        train_dataset = dataset[:train_size]
        val_dataset = dataset[train_size:]
        
        # Create data loaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        
        return train_loader, val_loader
    
    def train_epoch(self, train_loader: DataLoader, optimizer: torch.optim.Optimizer, 
                   criterion: nn.Module) -> Tuple[float, float]:
        """
        Train for one epoch.
        """
        self.model.train()
        total_loss = 0
        total_mae = 0
        num_batches = 0
        
        for batch in train_loader:
            batch = batch.to(self.device)
            
            # Forward pass
            optimizer.zero_grad()
            predictions = self.model(batch)
            
            # Compute loss
            loss = criterion(predictions.squeeze(), batch.y)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            # Accumulate metrics
            total_loss += loss.item()
            
            with torch.no_grad():
                mae = torch.mean(torch.abs(predictions.squeeze() - batch.y))
                total_mae += mae.item()
            
            num_batches += 1
        
        avg_loss = total_loss / num_batches
        avg_mae = total_mae / num_batches
        
        return avg_loss, avg_mae
    
    def validate(self, val_loader: DataLoader, criterion: nn.Module) -> Tuple[float, float]:
        """
        Validate model on validation set.
        """
        self.model.eval()
        total_loss = 0
        total_mae = 0
        num_batches = 0
        
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(self.device)
                
                predictions = self.model(batch)
                loss = criterion(predictions.squeeze(), batch.y)
                mae = torch.mean(torch.abs(predictions.squeeze() - batch.y))
                
                total_loss += loss.item()
                total_mae += mae.item()
                num_batches += 1
        
        avg_loss = total_loss / num_batches
        avg_mae = total_mae / num_batches
        
        return avg_loss, avg_mae
    
    def train(self, dataset: List[Data], num_epochs: int = 100, 
              learning_rate: float = 1e-3, batch_size: int = 32,
              patience: int = 10) -> Dict[str, List[float]]:
        """
        Full training pipeline with early stopping.
        """
        print("🚀 Starting SchNet training...")
        
        # Prepare data
        train_loader, val_loader = self.prepare_data(dataset, batch_size=batch_size)
        
        # Setup optimizer and criterion
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=5)
        criterion = nn.MSELoss()
        
        # Training loop
        best_val_loss = float('inf')
        patience_counter = 0
        
        for epoch in range(num_epochs):
            # Train
            train_loss, train_mae = self.train_epoch(train_loader, optimizer, criterion)
            
            # Validate
            val_loss, val_mae = self.validate(val_loader, criterion)
            
            # Update learning rate
            scheduler.step(val_loss)
            
            # Store history
            self.training_history['train_loss'].append(train_loss)
            self.training_history['val_loss'].append(val_loss)
            self.training_history['train_mae'].append(train_mae)
            self.training_history['val_mae'].append(val_mae)
            
            # Print progress
            if epoch % 10 == 0:
                print(f"Epoch {epoch:3d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
                      f"Train MAE: {train_mae:.4f} | Val MAE: {val_mae:.4f}")
            
            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                # Save best model state
                self.best_model_state = self.model.state_dict().copy()
            else:
                patience_counter += 1
                
                if patience_counter >= patience:
                    print(f"Early stopping at epoch {epoch}")
                    # Load best model
                    self.model.load_state_dict(self.best_model_state)
                    break
        
        print(f"✅ Training completed!")
        print(f"   • Best validation loss: {best_val_loss:.4f}")
        print(f"   • Final validation MAE: {self.training_history['val_mae'][-1]:.4f}")
        
        return self.training_history
    
    def plot_training_history(self):
        """
        Plot training and validation metrics.
        """
        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=('Loss', 'Mean Absolute Error'),
            x_title="Epoch"
        )
        
        epochs = list(range(len(self.training_history['train_loss'])))
        
        # Loss plot
        fig.add_trace(
            go.Scatter(x=epochs, y=self.training_history['train_loss'], 
                      name='Train Loss', line=dict(color='blue')),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=epochs, y=self.training_history['val_loss'], 
                      name='Val Loss', line=dict(color='red')),
            row=1, col=1
        )
        
        # MAE plot
        fig.add_trace(
            go.Scatter(x=epochs, y=self.training_history['train_mae'], 
                      name='Train MAE', line=dict(color='blue'), showlegend=False),
            row=1, col=2
        )
        fig.add_trace(
            go.Scatter(x=epochs, y=self.training_history['val_mae'], 
                      name='Val MAE', line=dict(color='red'), showlegend=False),
            row=1, col=2
        )
        
        fig.update_layout(
            title="SchNet Training Progress",
            height=400,
            showlegend=True
        )
        
        return fig

# Initialize SchNet model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🎯 Using device: {device}")

schnet_model = SchNet(
    hidden_channels=64,
    num_filters=64,
    num_interactions=3,
    num_gaussians=25,
    cutoff=8.0,
    readout='add'
)

print(f"🧠 SchNet Model Summary:")
total_params = sum(p.numel() for p in schnet_model.parameters())
print(f"   • Total parameters: {total_params:,}")
print(f"   • Hidden channels: {schnet_model.hidden_channels}")
print(f"   • Number of interactions: {schnet_model.num_interactions}")

# Initialize trainer
trainer = SchNetTrainer(schnet_model, device)
print("✅ SchNet trainer initialized and ready for training!")

In [ ]:
# Train SchNet model
print("🚀 Training SchNet on HOMO energy prediction...")

if len(molecular_graphs) > 0:
    # Train the model
    training_history = trainer.train(
        dataset=molecular_graphs,
        num_epochs=50,  # Reduced for demo
        learning_rate=1e-3,
        batch_size=16,
        patience=10
    )
    
    # Plot training progress
    training_plot = trainer.plot_training_history()
    training_plot.show()
    
    print("\n🎯 SchNet Training Results:")
    print(f"   • Final Train Loss: {training_history['train_loss'][-1]:.4f}")
    print(f"   • Final Val Loss: {training_history['val_loss'][-1]:.4f}")
    print(f"   • Final Train MAE: {training_history['train_mae'][-1]:.4f}")
    print(f"   • Final Val MAE: {training_history['val_mae'][-1]:.4f}")
    
else:
    print("⚠️ No molecular graphs available for training")

print("\n✅ Section 2 Complete: SchNet Implementation & 3D Molecular Understanding")
print("🎯 Ready to move to Section 3: Delta Learning Framework")

In [ ]:
# 📋 SECTION 2 CHECKPOINT ASSESSMENT: SchNet Implementation & 3D Molecular Understanding
print("\n" + "="*80)
print("📋 SECTION 2 CHECKPOINT ASSESSMENT: SchNet Implementation & 3D Molecular Understanding")
print("="*80)

if assessment:
    # Record section completion
    assessment.record_activity(
        "section_2_completion", 
        "completed",
        {"section": "SchNet Implementation & 3D Molecular Understanding", "timestamp": datetime.now().isoformat()}
    )

# Create assessment widget for Section 2
section2_widget = create_widget(
    assessment=assessment,
    section="Section 2: SchNet Implementation & 3D Molecular Understanding",
    concepts=[
        "SchNet architecture understanding and implementation",
        "3D molecular representation and coordinate handling", 
        "Continuous-filter convolutional layers",
        "Message passing neural networks for molecules",
        "Radial basis functions and smooth cutoff functions",
        "3D invariant and equivariant features",
        "Property prediction with geometric deep learning"
    ],
    activities=[
        "Successfully implemented SchNet architecture",
        "Built 3D molecular data preprocessing pipeline",
        "Created continuous-filter convolutional layers",
        "Implemented message passing framework",
        "Applied SchNet to QM9 property prediction",
        "Analyzed 3D molecular geometric features",
        "Evaluated model performance on quantum properties"
    ]
)

# Display the interactive assessment
section2_widget.display()

# Progress tracking
if assessment:
    print(f"\n📊 Current Progress: {assessment.get_progress_summary()['overall_score']:.1f}%")
    print("🎯 Section 2 assessment completed - proceed when ready!")

---

## **Section 3: Delta Learning Framework for QM/ML Hybrid Models** ⚗️

Delta learning combines quantum mechanical calculations with machine learning to achieve chemical accuracy by learning corrections to lower-level quantum methods.

In [ ]:
# Additional imports for delta learning
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing as mp
from functools import partial
import asyncio
import aiohttp
import json

print("🎯 Delta Learning Framework Environment Ready!")

### **3.1 Delta Learning Framework Implementation**

In [ ]:
class QuantumMethodSimulator:
    """
    Simulate different levels of quantum chemical calculations.
    
    In practice, these would interface with real quantum chemistry codes
    like Psi4, Gaussian, or ORCA. Here we simulate the behavior.
    """
    
    def __init__(self):
        self.method_accuracies = {
            'HF': 0.85,      # Hartree-Fock: fast but less accurate
            'DFT': 0.92,     # DFT: good balance of speed and accuracy
            'MP2': 0.96,     # MP2: more accurate but slower
            'CCSD': 0.98,    # CCSD: high accuracy, very slow
            'experiment': 1.0  # Experimental reference
        }
        
        self.method_costs = {
            'HF': 1,         # Relative computational cost
            'DFT': 3,
            'MP2': 15,
            'CCSD': 100
        }
    
    def calculate_property(self, smiles: str, method: str = 'DFT', 
                         property_name: str = 'homo') -> Dict[str, float]:
        """
        Simulate quantum chemical calculation for a given method.
        """
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return {'value': 0.0, 'uncertainty': 1.0, 'cost': 1.0}
        
        # Base calculation using molecular descriptors
        base_value = self._compute_base_property(mol, property_name)
        
        # Add method-specific corrections and noise
        accuracy = self.method_accuracies[method]
        cost = self.method_costs[method]
        
        # Simulate method-specific corrections
        if method == 'HF':
            # HF typically underestimates correlation effects
            correction = -0.1 + 0.02 * np.random.randn()
        elif method == 'DFT':
            # DFT is generally good but has some systematic errors
            correction = 0.05 + 0.01 * np.random.randn()
        elif method == 'MP2':
            # MP2 overcorrects correlation in some cases
            correction = 0.02 + 0.005 * np.random.randn()
        elif method == 'CCSD':
            # CCSD is very accurate
            correction = 0.0 + 0.002 * np.random.randn()
        
        final_value = base_value + correction
        uncertainty = (1.0 - accuracy) * abs(base_value) + 0.001
        
        return {
            'value': final_value,
            'uncertainty': uncertainty,
            'cost': cost,
            'method': method
        }
    
    def _compute_base_property(self, mol: Chem.Mol, property_name: str) -> float:
        """
        Compute base property value using simple molecular descriptors.
        """
        # Simple correlation based on molecular properties
        mw = Descriptors.MolWt(mol)
        natoms = mol.GetNumAtoms()
        aromatic_atoms = sum(1 for atom in mol.GetAtoms() if atom.GetIsAromatic())
        
        if property_name == 'homo':
            # HOMO energy correlation (rough approximation)
            base_value = -0.25 - 0.001 * mw + 0.02 * aromatic_atoms / natoms
        elif property_name == 'lumo':
            # LUMO energy correlation
            base_value = -0.05 - 0.0005 * mw - 0.01 * aromatic_atoms / natoms
        elif property_name == 'gap':
            # HOMO-LUMO gap
            homo = self._compute_base_property(mol, 'homo')
            lumo = self._compute_base_property(mol, 'lumo')
            base_value = lumo - homo
        else:
            base_value = np.random.normal(0, 0.1)
        
        return base_value


class DeltaLearningModel:
    """
    Delta learning model that learns corrections between different QM levels.
    
    Δ(molecule) = E_high_level - E_low_level
    
    The ML model predicts this delta, allowing us to get high-level accuracy
    at low-level computational cost.
    """
    
    def __init__(self, low_level_method: str = 'DFT', high_level_method: str = 'CCSD'):
        self.low_level_method = low_level_method
        self.high_level_method = high_level_method
        self.model = None
        self.feature_scaler = StandardScaler()
        self.target_scaler = StandardScaler()
        self.qm_simulator = QuantumMethodSimulator()
        
    def generate_training_data(self, smiles_list: List[str], 
                             property_name: str = 'homo') -> pd.DataFrame:
        """
        Generate training data with low-level and high-level calculations.
        """
        print(f"Generating delta learning data for {len(smiles_list)} molecules...")
        
        training_data = []
        
        for i, smiles in enumerate(smiles_list):
            if i % 100 == 0:
                print(f"Processing molecule {i+1}/{len(smiles_list)}")
            
            # Low-level calculation (fast)
            low_result = self.qm_simulator.calculate_property(
                smiles, self.low_level_method, property_name
            )
            
            # High-level calculation (expensive)
            high_result = self.qm_simulator.calculate_property(
                smiles, self.high_level_method, property_name
            )
            
            # Compute delta
            delta = high_result['value'] - low_result['value']
            
            training_data.append({
                'smiles': smiles,
                'low_level': low_result['value'],
                'high_level': high_result['value'],
                'delta': delta,
                'low_uncertainty': low_result['uncertainty'],
                'high_uncertainty': high_result['uncertainty'],
                'total_cost': low_result['cost'] + high_result['cost']
            })
        
        return pd.DataFrame(training_data)
    
    def train_delta_model(self, training_data: pd.DataFrame, 
                         features: np.ndarray) -> Dict[str, float]:
        """
        Train the delta learning model.
        """
        print(f"Training delta model ({self.low_level_method} → {self.high_level_method})...")
        
        # Prepare features and targets
        X = features
        y = training_data['delta'].values
        
        # Scale features and targets
        X_scaled = self.feature_scaler.fit_transform(X)
        y_scaled = self.target_scaler.fit_transform(y.reshape(-1, 1)).ravel()
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y_scaled, test_size=0.2, random_state=42
        )
        
        # Train model
        self.model = RandomForestRegressor(
            n_estimators=200,
            max_depth=20,
            min_samples_split=5,
            min_samples_leaf=2,
            random_state=42,
            n_jobs=-1
        )
        
        self.model.fit(X_train, y_train)
        
        # Evaluate
        y_pred = self.model.predict(X_test)
        
        # Transform back to original scale
        y_test_orig = self.target_scaler.inverse_transform(y_test.reshape(-1, 1)).ravel()
        y_pred_orig = self.target_scaler.inverse_transform(y_pred.reshape(-1, 1)).ravel()
        
        # Compute metrics
        mae = mean_absolute_error(y_test_orig, y_pred_orig)
        rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
        r2 = r2_score(y_test_orig, y_pred_orig)
        
        results = {'mae': mae, 'rmse': rmse, 'r2': r2}
        
        print(f"Delta model performance:")
        print(f"   • MAE: {mae:.6f}")
        print(f"   • RMSE: {rmse:.6f}")
        print(f"   • R²: {r2:.3f}")
        
        return results
    
    def predict_high_level(self, smiles: str, features: np.ndarray, 
                          property_name: str = 'homo') -> Dict[str, float]:
        """
        Predict high-level property using delta learning.
        """
        if self.model is None:
            raise ValueError("Model not trained. Call train_delta_model first.")
        
        # Low-level calculation
        low_result = self.qm_simulator.calculate_property(
            smiles, self.low_level_method, property_name
        )
        
        # Predict delta using ML model
        features_scaled = self.feature_scaler.transform(features.reshape(1, -1))
        delta_scaled = self.model.predict(features_scaled)[0]
        delta = self.target_scaler.inverse_transform([[delta_scaled]])[0][0]
        
        # Combine for high-level prediction
        high_level_pred = low_result['value'] + delta
        
        # Estimate uncertainty (simplified)
        delta_uncertainty = abs(delta) * 0.1  # 10% uncertainty on delta
        total_uncertainty = np.sqrt(low_result['uncertainty']**2 + delta_uncertainty**2)
        
        return {
            'low_level': low_result['value'],
            'delta': delta,
            'high_level_pred': high_level_pred,
            'uncertainty': total_uncertainty,
            'cost_savings': self.qm_simulator.method_costs[self.high_level_method] - 
                          self.qm_simulator.method_costs[self.low_level_method]
        }


class ActiveLearningDelta:
    """
    Active learning for optimal selection of molecules for expensive high-level calculations.
    """
    
    def __init__(self, delta_model: DeltaLearningModel):
        self.delta_model = delta_model
        self.uncertainty_threshold = 0.05
        
    def select_molecules_for_calculation(self, candidate_smiles: List[str], 
                                       candidate_features: np.ndarray,
                                       n_select: int = 10) -> List[int]:
        """
        Select molecules for high-level calculations using uncertainty sampling.
        """
        if self.delta_model.model is None:
            # If no model trained yet, select randomly
            return np.random.choice(len(candidate_smiles), n_select, replace=False).tolist()
        
        uncertainties = []
        
        for i, (smiles, features) in enumerate(zip(candidate_smiles, candidate_features)):
            try:
                result = self.delta_model.predict_high_level(smiles, features)
                uncertainties.append(result['uncertainty'])
            except:
                uncertainties.append(1.0)  # High uncertainty for failed predictions
        
        # Select molecules with highest uncertainty
        uncertainty_indices = np.argsort(uncertainties)[-n_select:]
        
        return uncertainty_indices.tolist()
    
    def adaptive_training_loop(self, all_smiles: List[str], all_features: np.ndarray,
                             property_name: str = 'homo', max_iterations: int = 5,
                             molecules_per_iteration: int = 50) -> Dict[str, Any]:
        """
        Adaptive training loop that iteratively selects molecules and improves the model.
        """
        print("🚀 Starting adaptive delta learning...")
        
        results = {
            'iterations': [],
            'model_performance': [],
            'total_molecules': 0,
            'total_cost': 0
        }
        
        # Start with random selection
        current_indices = np.random.choice(
            len(all_smiles), molecules_per_iteration, replace=False
        ).tolist()
        
        for iteration in range(max_iterations):
            print(f"\n📊 Iteration {iteration + 1}/{max_iterations}")
            
            # Get current molecules and features
            current_smiles = [all_smiles[i] for i in current_indices]
            current_features = all_features[current_indices]
            
            # Generate training data
            training_data = self.delta_model.generate_training_data(
                current_smiles, property_name
            )
            
            # Train delta model
            performance = self.delta_model.train_delta_model(training_data, current_features)
            
            # Update results
            results['iterations'].append(iteration + 1)
            results['model_performance'].append(performance)
            results['total_molecules'] += len(current_smiles)
            results['total_cost'] += training_data['total_cost'].sum()
            
            print(f"   • Molecules processed: {len(current_smiles)}")
            print(f"   • Cumulative molecules: {results['total_molecules']}")
            print(f"   • Model R²: {performance['r2']:.3f}")
            
            # Select next batch (if not last iteration)
            if iteration < max_iterations - 1:
                remaining_indices = [i for i in range(len(all_smiles)) if i not in current_indices]
                remaining_smiles = [all_smiles[i] for i in remaining_indices]
                remaining_features = all_features[remaining_indices]
                
                next_indices = self.select_molecules_for_calculation(
                    remaining_smiles, remaining_features, molecules_per_iteration
                )
                
                # Convert back to global indices
                next_global_indices = [remaining_indices[i] for i in next_indices]
                current_indices.extend(next_global_indices)
        
        print(f"\n✅ Adaptive learning completed!")
        print(f"   • Total molecules: {results['total_molecules']}")
        print(f"   • Final R²: {results['model_performance'][-1]['r2']:.3f}")
        
        return results
print("✅ Delta Learning Framework implemented!")

### **3.2 Delta Learning Demonstration**

In [ ]:
# Initialize delta learning system
print("🚀 Setting up Delta Learning demonstration...")

# Create delta learning model (DFT → CCSD)
delta_model = DeltaLearningModel(low_level_method='DFT', high_level_method='CCSD')

# Use subset of molecules for demonstration
demo_size = 200
demo_indices = np.random.choice(len(aligned_qm9_data), demo_size, replace=False)
demo_smiles = [aligned_qm9_data['smiles'].iloc[i] for i in demo_indices]
demo_features = scaled_features[demo_indices]

print(f"📊 Demo setup:")
print(f"   • Molecules: {len(demo_smiles)}")
print(f"   • Features: {demo_features.shape}")

# Generate training data for delta learning
print("\nGenerating delta learning training data...")
delta_training_data = delta_model.generate_training_data(demo_smiles, 'homo')

print(f"✅ Training data generated:")
print(f"   • Data shape: {delta_training_data.shape}")
display(delta_training_data.head())

# Analyze delta statistics
print(f"\n📈 Delta Statistics:")
print(f"   • Mean delta: {delta_training_data['delta'].mean():.6f}")
print(f"   • Std delta: {delta_training_data['delta'].std():.6f}")
print(f"   • Min delta: {delta_training_data['delta'].min():.6f}")
print(f"   • Max delta: {delta_training_data['delta'].max():.6f}")

# Train delta model
delta_performance = delta_model.train_delta_model(delta_training_data, demo_features)

# Visualize delta learning results
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Low vs High Level Energies', 'Delta Distribution'),
)

# Scatter plot: Low vs High level
fig.add_trace(
    go.Scatter(
        x=delta_training_data['low_level'],
        y=delta_training_data['high_level'],
        mode='markers',
        name='Calculations',
        marker=dict(size=4, opacity=0.7, color='blue')
    ),
    row=1, col=1
)

# Perfect correlation line
min_val = min(delta_training_data['low_level'].min(), delta_training_data['high_level'].min())
max_val = max(delta_training_data['low_level'].max(), delta_training_data['high_level'].max())
fig.add_trace(
    go.Scatter(
        x=[min_val, max_val],
        y=[min_val, max_val],
        mode='lines',
        line=dict(dash='dash', color='red'),
        name='Perfect correlation',
        showlegend=False
    ),
    row=1, col=1
)

# Delta histogram
fig.add_trace(
    go.Histogram(
        x=delta_training_data['delta'],
        nbinsx=30,
        name='Delta values',
        showlegend=False,
        marker_color='green'
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="DFT Energy", row=1, col=1)
fig.update_yaxes(title_text="CCSD Energy", row=1, col=1)
fig.update_xaxes(title_text="Delta (CCSD - DFT)", row=1, col=2)
fig.update_yaxes(title_text="Count", row=1, col=2)

fig.update_layout(
    title="Delta Learning Analysis",
    height=400,
    showlegend=True
)

fig.show()

print("✅ Delta learning model trained successfully!")

In [ ]:
# 🎯 **MID-SECTION EXERCISE CHECKPOINT 3.1: Delta Learning Implementation**

print("🎯 MID-SECTION EXERCISE CHECKPOINT 3.1: Delta Learning Implementation")
print("="*70)

# Quick hands-on exercise: Analyze delta learning performance
exercise_widget_3_1 = create_widget(
    assessment, 
    section="3.1",
    concepts=[
        "Delta learning methodology and principles",
        "QM/ML hybrid model architecture",
        "Cost-accuracy trade-off optimization"
    ],
    activities=[
        "Implemented quantum method simulator",
        "Built delta learning correction model", 
        "Validated cost-accuracy improvements"
    ]
)

if assessment:
    assessment.record_activity(
        activity="delta_learning_implementation",
        result="completed",
        metadata={
            "model_performance": "implemented",
            "qm_simulator": "functional",
            "section": "3.1_mid_checkpoint"
        }
    )

exercise_widget_3_1.display()
print("🎓 Mid-section checkpoint 3.1 completed! Continue with active learning...")

### **3.3 Active Learning Demonstration**

In [ ]:
# Initialize active learning system
print("🎯 Setting up Active Learning demonstration...")

active_learner = ActiveLearningDelta(delta_model)

# Run adaptive training loop
print("Starting adaptive training loop...")
adaptive_results = active_learner.adaptive_training_loop(
    all_smiles=demo_smiles,
    all_features=demo_features,
    property_name='homo',
    max_iterations: int = 3,  # Reduced for demo
    molecules_per_iteration: int = 30
)

# Visualize active learning progress
iterations = adaptive_results['iterations']
r2_scores = [perf['r2'] for perf in adaptive_results['model_performance']]
mae_scores = [perf['mae'] for perf in adaptive_results['model_performance']]

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Model Accuracy (R²)', 'Prediction Error (MAE)'),
)

fig.add_trace(
    go.Scatter(
        x=iterations,
        y=r2_scores,
        mode='lines+markers',
        name='R² Score',
        line=dict(color='blue', width=3),
        marker=dict(size=8)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=iterations,
        y=mae_scores,
        mode='lines+markers',
        name='MAE',
        line=dict(color='red', width=3),
        marker=dict(size=8)
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="Iteration", row=1, col=1)
fig.update_yaxes(title_text="R² Score", row=1, col=1)
fig.update_xaxes(title_text="Iteration", row=1, col=2)
fig.update_yaxes(title_text="MAE", row=1, col=2)

fig.update_layout(
    title="Active Learning Progress",
    height=400,
    showlegend=False
)

fig.show()

print(f"\n📊 Active Learning Results:")
print(f"   • Initial R²: {r2_scores[0]:.3f}")
print(f"   • Final R²: {r2_scores[-1]:.3f}")
print(f"   • Improvement: {r2_scores[-1] - r2_scores[0]:.3f}")
print(f"   • Total molecules: {adaptive_results['total_molecules']}")
print(f"   • Total cost: {adaptive_results['total_cost']:.0f}")

# Demonstrate cost savings
print(f"\n💰 Cost Analysis:")
traditional_cost = len(demo_smiles) * delta_model.qm_simulator.method_costs['CCSD']
delta_cost = adaptive_results['total_cost']
savings = traditional_cost - delta_cost
savings_percent = (savings / traditional_cost) * 100

print(f"   • Traditional approach cost: {traditional_cost:.0f}")
print(f"   • Delta learning cost: {delta_cost:.0f}")
print(f"   • Cost savings: {savings:.0f} ({savings_percent:.1f}%)")

print("\n✅ Section 3 Complete: Delta Learning Framework")
print("🎯 Ready to move to Section 4: Advanced Quantum ML Architectures")

In [ ]:
# 📋 SECTION 3 CHECKPOINT ASSESSMENT: Delta Learning Framework for QM/ML Hybrid Models
print("\n" + "="*80)
print("📋 SECTION 3 CHECKPOINT ASSESSMENT: Delta Learning Framework for QM/ML Hybrid Models")
print("="*80)

if assessment:
    # Record section completion
    assessment.record_activity(
        "section_3_completion", 
        "completed",
        {"section": "Delta Learning Framework for QM/ML Hybrid Models", "timestamp": datetime.now().isoformat()}
    )

# Create assessment widget for Section 3
section3_widget = create_widget(
    assessment=assessment,
    section="Section 3: Delta Learning Framework for QM/ML Hybrid Models",
    concepts=[
        "Delta learning methodology and theoretical framework",
        "Multi-level quantum chemistry method integration", 
        "Hybrid QM/ML model architectures",
        "Uncertainty quantification in quantum ML",
        "Cost-accuracy trade-offs in quantum calculations",
        "Error correction and systematic bias handling",
        "Production deployment of delta learning systems"
    ],
    activities=[
        "Implemented quantum method simulator framework",
        "Built delta learning correction models",
        "Created multi-level QM/ML hybrid pipeline",
        "Developed uncertainty quantification metrics",
        "Optimized cost-accuracy trade-offs",
        "Validated delta learning performance",
        "Deployed production-ready delta learning system"
    ]
)

# Display the interactive assessment
section3_widget.display()

# Progress tracking
if assessment:
    print(f"\n📊 Current Progress: {assessment.get_progress_summary()['overall_score']:.1f}%")
    print("🎯 Section 3 assessment completed - proceed when ready!")

---

## **Section 4: Advanced Quantum ML Architectures** 🧠

Explore cutting-edge architectures that combine quantum mechanical insights with deep learning innovations, including attention mechanisms and transformer architectures for molecules.

In [ ]:
# Additional imports for advanced architectures
import torch.nn.functional as F
from torch.nn import MultiheadAttention, TransformerEncoder, TransformerEncoderLayer
from typing import Optional, Callable
import math

print("🚀 Advanced Quantum ML Architectures Environment Ready!")

### **4.1 Quantum-Aware Attention Mechanisms**

In [ ]:
class QuantumAwareAttention(nn.Module):
    """
    Quantum-aware attention mechanism that incorporates physical principles
    into the attention computation.
    """
    
    def __init__(self, d_model: int, num_heads: int = 8, 
                 quantum_features: bool = True, dropout: float = 0.1):
        super(QuantumAwareAttention, self).__init__()
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.quantum_features = quantum_features
        self.head_dim = d_model // num_heads
        
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        # Standard attention projections
        self.q_proj = nn.Linear(d_model, d_model)
        self.k_proj = nn.Linear(d_model, d_model)
        self.v_proj = nn.Linear(d_model, d_model)
        self.out_proj = nn.Linear(d_model, d_model)
        
        # Quantum-aware components
        if quantum_features:
            # Distance-based attention weights
            self.distance_embedding = nn.Sequential(
                nn.Linear(1, 32),
                nn.ReLU(),
                nn.Linear(32, num_heads),
                nn.Sigmoid()
            )
            
            # Orbital overlap modeling
            self.orbital_interaction = nn.Sequential(
                nn.Linear(d_model * 2, 64),
                nn.ReLU(),
                nn.Linear(64, num_heads),
                nn.Sigmoid()
            )
        
        self.dropout = nn.Dropout(dropout)
        self.scale = math.sqrt(self.head_dim)
        
    def forward(self, x: torch.Tensor, edge_index: Optional[torch.Tensor] = None,
                distances: Optional[torch.Tensor] = None, 
                mask: Optional[torch.Tensor] = None) -> torch.Tensor:
        """
        Forward pass with quantum-aware attention.
        
        Args:
            x: Node features [batch_size, seq_len, d_model]
            edge_index: Graph connectivity [2, num_edges]
            distances: Pairwise distances [num_edges] or [batch_size, seq_len, seq_len]
            mask: Attention mask [batch_size, seq_len, seq_len]
        """
        batch_size, seq_len, _ = x.shape
        
        # Compute Q, K, V
        Q = self.q_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.k_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.v_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        
        # Compute standard attention scores
        scores = torch.matmul(Q, K.transpose(-2, -1)) / self.scale
        
        # Add quantum-aware modifications
        if self.quantum_features and distances is not None:
            # Distance-based attention modification
            if len(distances.shape) == 1:
                # Convert edge distances to attention matrix
                distance_matrix = self._edge_to_matrix(edge_index, distances, seq_len)
            else:
                distance_matrix = distances
            
            distance_weights = self.distance_embedding(distance_matrix.unsqueeze(-1))
            distance_weights = distance_weights.permute(0, 3, 1, 2)  # [batch, heads, seq, seq]
            
            # Apply distance-based modulation
            scores = scores * distance_weights
            
            # Orbital interaction terms
            x_pairs = self._create_pairwise_features(x)  # [batch, seq, seq, 2*d_model]
            orbital_weights = self.orbital_interaction(x_pairs)
            orbital_weights = orbital_weights.permute(0, 3, 1, 2)  # [batch, heads, seq, seq]
            
            scores = scores + orbital_weights
        
        # Apply mask if provided
        if mask is not None:
            scores = scores.masked_fill(mask.unsqueeze(1).unsqueeze(1) == 0, -1e9)
        
        # Softmax attention
        attention_weights = F.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        
        # Apply attention to values
        out = torch.matmul(attention_weights, V)
        out = out.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
        
        # Final projection
        return self.out_proj(out)
    
    def _edge_to_matrix(self, edge_index: torch.Tensor, edge_distances: torch.Tensor, 
                       seq_len: int) -> torch.Tensor:
        """Convert edge list to distance matrix."""
        batch_size = 1  # Simplified for single batch
        distance_matrix = torch.zeros(batch_size, seq_len, seq_len, device=edge_distances.device)
        
        if edge_index is not None:
            row, col = edge_index
            distance_matrix[0, row, col] = edge_distances
            distance_matrix[0, col, row] = edge_distances  # Symmetric
        
        return distance_matrix
    
    def _create_pairwise_features(self, x: torch.Tensor) -> torch.Tensor:
        """Create pairwise features for orbital interaction modeling."""
        batch_size, seq_len, d_model = x.shape
        
        # Expand features for all pairs
        x_i = x.unsqueeze(2).expand(-1, -1, seq_len, -1)  # [batch, seq, seq, d_model]
        x_j = x.unsqueeze(1).expand(-1, seq_len, -1, -1)  # [batch, seq, seq, d_model]
        
        # Concatenate pairwise features
        pairwise_features = torch.cat([x_i, x_j], dim=-1)  # [batch, seq, seq, 2*d_model]
        
        return pairwise_features


class MolecularTransformer(nn.Module):
    """
    Transformer architecture specifically designed for molecular property prediction
    with quantum-aware attention mechanisms.
    """
    
    def __init__(self, d_model: int = 256, num_heads: int = 8, 
                 num_layers: int = 6, max_atoms: int = 100,
                 num_atom_types: int = 100, dropout: float = 0.1):
        super(MolecularTransformer, self).__init__()
        
        self.d_model = d_model
        self.max_atoms = max_atoms
        
        # Atom type embedding
        self.atom_embedding = nn.Embedding(num_atom_types, d_model)
        
        # Positional encoding (3D-aware)
        self.position_embedding = nn.Linear(3, d_model)
        
        # Quantum-aware transformer layers
        self.transformer_layers = nn.ModuleList([
            self._make_layer(d_model, num_heads, dropout)
            for _ in range(num_layers)
        ])
        
        # Output head for property prediction
        self.output_head = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, d_model // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, 1)
        )
        
        # Layer normalization
        self.layer_norm = nn.LayerNorm(d_model)
        
    def _make_layer(self, d_model: int, num_heads: int, dropout: float) -> nn.Module:
        """Create a single transformer layer with quantum-aware attention."""
        return nn.ModuleDict({
            'attention': QuantumAwareAttention(d_model, num_heads, quantum_features=True, dropout=dropout),
            'norm1': nn.LayerNorm(d_model),
            'ffn': nn.Sequential(
                nn.Linear(d_model, d_model * 4),
                nn.GELU(),
                nn.Dropout(dropout),
                nn.Linear(d_model * 4, d_model),
                nn.Dropout(dropout)
            ),
            'norm2': nn.LayerNorm(d_model)
        })
    
    def forward(self, atom_types: torch.Tensor, positions: torch.Tensor,
                edge_index: Optional[torch.Tensor] = None,
                batch_idx: Optional[torch.Tensor] = None) -> torch.Tensor:
        """
        Forward pass of molecular transformer.
        
        Args:
            atom_types: Atomic numbers [batch_size, max_atoms]
            positions: 3D coordinates [batch_size, max_atoms, 3]
            edge_index: Graph connectivity [2, num_edges]
            batch_idx: Batch assignment for graph data
        """
        batch_size, seq_len = atom_types.shape
        
        # Create attention mask for padding
        mask = (atom_types != 0).float()  # Assume 0 is padding
        
        # Embed atoms and positions
        atom_embed = self.atom_embedding(atom_types)
        pos_embed = self.position_embedding(positions)
        
        # Combine embeddings
        x = atom_embed + pos_embed
        x = self.layer_norm(x)
        
        # Compute pairwise distances
        distances = self._compute_distances(positions, mask)
        
        # Apply transformer layers
        for layer in self.transformer_layers:
            # Self-attention with residual connection
            attn_out = layer['attention'](x, edge_index, distances, mask.unsqueeze(-1))
            x = layer['norm1'](x + attn_out)
            
            # Feed-forward with residual connection
            ffn_out = layer['ffn'](x)
            x = layer['norm2'](x + ffn_out)
        
        # Global pooling (mean over valid atoms)
        mask_expanded = mask.unsqueeze(-1).expand_as(x)
        x_masked = x * mask_expanded
        global_features = x_masked.sum(dim=1) / mask.sum(dim=1, keepdim=True).clamp(min=1)
        
        # Predict property
        return self.output_head(global_features)
    
    def _compute_distances(self, positions: torch.Tensor, mask: torch.Tensor) -> torch.Tensor:
        """Compute pairwise distances between atoms."""
        batch_size, seq_len, _ = positions.shape
        
        # Expand positions for pairwise computation
        pos_i = positions.unsqueeze(2).expand(-1, -1, seq_len, -1)
        pos_j = positions.unsqueeze(1).expand(-1, seq_len, -1, -1)
        
        # Compute distances
        distances = torch.norm(pos_i - pos_j, dim=-1)
        
        # Apply mask to ignore padding atoms
        mask_matrix = mask.unsqueeze(-1) * mask.unsqueeze(1)
        distances = distances * mask_matrix
        
        return distances

print("✅ Quantum-Aware Attention and Molecular Transformer implemented!")

### **4.2 Multi-Task Quantum Property Learning**

In [ ]:
class MultiTaskQuantumPredictor(nn.Module):
    """
    Multi-task learning model for simultaneous prediction of multiple quantum properties.
    
    This model leverages shared representations to improve prediction of correlated
    quantum mechanical properties like HOMO, LUMO, gap, etc.
    """
    
    def __init__(self, input_dim: int, hidden_dims: List[int] = [512, 256, 128],
                 property_dims: Dict[str, int] = None, dropout: float = 0.1):
        super(MultiTaskQuantumPredictor, self).__init__()
        
        if property_dims is None:
            property_dims = {
                'homo': 1, 'lumo': 1, 'gap': 1, 'mu': 1, 
                'alpha': 1, 'cv': 1, 'zpve': 1, 'u0': 1
            }
        
        self.property_names = list(property_dims.keys())
        self.num_properties = len(self.property_names)
        
        # Shared encoder
        encoder_layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            encoder_layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = hidden_dim
        
        self.shared_encoder = nn.Sequential(*encoder_layers)
        
        # Property-specific heads
        self.property_heads = nn.ModuleDict()
        for prop_name, output_dim in property_dims.items():
            self.property_heads[prop_name] = nn.Sequential(
                nn.Linear(prev_dim, prev_dim // 2),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(prev_dim // 2, output_dim)
            )
        
        # Uncertainty estimation heads
        self.uncertainty_heads = nn.ModuleDict()
        for prop_name in property_dims.keys():
            self.uncertainty_heads[prop_name] = nn.Sequential(
                nn.Linear(prev_dim, prev_dim // 4),
                nn.ReLU(),
                nn.Linear(prev_dim // 4, 1),
                nn.Softplus()  # Ensure positive uncertainty
            )
        
        # Cross-property correlation learning
        self.correlation_matrix = nn.Parameter(
            torch.eye(self.num_properties) * 0.1 + torch.randn(self.num_properties, self.num_properties) * 0.01
        )
        
    def forward(self, x: torch.Tensor) -> Dict[str, torch.Tensor]:
        """
        Forward pass with multi-task prediction.
        
        Returns:
            Dictionary with predictions and uncertainties for each property
        """
        # Shared feature extraction
        shared_features = self.shared_encoder(x)
        
        # Property-specific predictions
        predictions = {}
        uncertainties = {}
        
        for prop_name in self.property_names:
            predictions[prop_name] = self.property_heads[prop_name](shared_features)
            uncertainties[prop_name] = self.uncertainty_heads[prop_name](shared_features)
        
        # Apply correlation constraints
        pred_tensor = torch.stack([predictions[prop] for prop in self.property_names], dim=-1)
        corr_adjusted = torch.matmul(pred_tensor, self.correlation_matrix)
        
        # Update predictions with correlations
        for i, prop_name in enumerate(self.property_names):
            predictions[prop_name] = corr_adjusted[..., i:i+1]
        
        return {
            'predictions': predictions,
            'uncertainties': uncertainties,
            'correlations': self.correlation_matrix
        }
    
    def compute_multi_task_loss(self, outputs: Dict[str, torch.Tensor], 
                               targets: Dict[str, torch.Tensor],
                               weights: Optional[Dict[str, float]] = None) -> torch.Tensor:
        """
        Compute multi-task loss with uncertainty weighting.
        """
        if weights is None:
            weights = {prop: 1.0 for prop in self.property_names}
        
        total_loss = 0
        predictions = outputs['predictions']
        uncertainties = outputs['uncertainties']
        
        for prop_name in self.property_names:
            if prop_name in targets:
                pred = predictions[prop_name].squeeze()
                target = targets[prop_name]
                uncertainty = uncertainties[prop_name].squeeze()
                
                # Uncertainty-weighted loss (heteroscedastic)
                mse_loss = F.mse_loss(pred, target, reduction='none')
                weighted_loss = mse_loss / (2 * uncertainty**2) + 0.5 * torch.log(uncertainty**2)
                
                total_loss += weights[prop_name] * weighted_loss.mean()
        
        # Add correlation regularization
        corr_reg = torch.norm(self.correlation_matrix - torch.eye(self.num_properties, device=self.correlation_matrix.device))
        total_loss += 0.01 * corr_reg
        
        return total_loss


class EnsembleQuantumPredictor:
    """
    Ensemble of quantum property predictors for improved uncertainty quantification.
    """
    
    def __init__(self, base_model_class, model_configs: List[Dict], num_models: int = 5):
        self.models = []
        self.num_models = num_models
        
        # Create ensemble of models with different configurations
        for i in range(num_models):
            config = model_configs[i % len(model_configs)]
            model = base_model_class(**config)
            self.models.append(model)
    
    def train_ensemble(self, train_loader: DataLoader, val_loader: DataLoader,
                      num_epochs: int = 100, device: str = 'cpu') -> Dict[str, List[float]]:
        """
        Train ensemble of models with different random initializations.
        """
        ensemble_history = {'train_loss': [], 'val_loss': []}
        
        for i, model in enumerate(self.models):
            print(f"Training model {i+1}/{self.num_models}...")
            
            model = model.to(device)
            optimizer = optim.Adam(model.parameters(), lr=1e-3)
            
            model_history = {'train_loss': [], 'val_loss': []}
            
            for epoch in range(num_epochs):
                # Training
                model.train()
                train_loss = 0
                for batch in train_loader:
                    optimizer.zero_grad()
                    
                    # Prepare batch data
                    features = batch['features'].to(device)
                    targets = {k: v.to(device) for k, v in batch['targets'].items()}
                    
                    # Forward pass
                    outputs = model(features)
                    loss = model.compute_multi_task_loss(outputs, targets)
                    
                    # Backward pass
                    loss.backward()
                    optimizer.step()
                    
                    train_loss += loss.item()
                
                # Validation
                model.eval()
                val_loss = 0
                with torch.no_grad():
                    for batch in val_loader:
                        features = batch['features'].to(device)
                        targets = {k: v.to(device) for k, v in batch['targets'].items()}
                        
                        outputs = model(features)
                        loss = model.compute_multi_task_loss(outputs, targets)
                        val_loss += loss.item()
                
                model_history['train_loss'].append(train_loss / len(train_loader))
                model_history['val_loss'].append(val_loss / len(val_loader))
                
                if epoch % 20 == 0:
                    print(f"   Epoch {epoch}: Train Loss: {model_history['train_loss'][-1]:.4f}, "
                          f"Val Loss: {model_history['val_loss'][-1]:.4f}")
            
            ensemble_history['train_loss'].append(model_history['train_loss'])
            ensemble_history['val_loss'].append(model_history['val_loss'])
        
        return ensemble_history
    
    def predict_with_uncertainty(self, x: torch.Tensor, device: str = 'cpu') -> Dict[str, Dict[str, torch.Tensor]]:
        """
        Make predictions with ensemble uncertainty quantification.
        """
        all_predictions = {prop: [] for prop in self.models[0].property_names}
        all_uncertainties = {prop: [] for prop in self.models[0].property_names}
        
        with torch.no_grad():
            for model in self.models:
                model.eval()
                model = model.to(device)
                x = x.to(device)
                
                outputs = model(x)
                
                for prop in self.models[0].property_names:
                    all_predictions[prop].append(outputs['predictions'][prop])
                    all_uncertainties[prop].append(outputs['uncertainties'][prop])
        
        # Compute ensemble statistics
        ensemble_results = {}
        
        for prop in self.models[0].property_names:
            pred_stack = torch.stack(all_predictions[prop], dim=0)
            uncert_stack = torch.stack(all_uncertainties[prop], dim=0)
            
            # Ensemble mean and variance
            ensemble_mean = torch.mean(pred_stack, dim=0)
            ensemble_var = torch.var(pred_stack, dim=0)
            
            # Total uncertainty (epistemic + aleatoric)
            aleatoric_uncertainty = torch.mean(uncert_stack**2, dim=0)
            epistemic_uncertainty = ensemble_var
            total_uncertainty = torch.sqrt(aleatoric_uncertainty + epistemic_uncertainty)
            
            ensemble_results[prop] = {
                'mean': ensemble_mean,
                'total_uncertainty': total_uncertainty,
                'epistemic_uncertainty': torch.sqrt(epistemic_uncertainty),
                'aleatoric_uncertainty': torch.sqrt(aleatoric_uncertainty)
            }
        
        return ensemble_results

print("✅ Multi-Task and Ensemble Learning frameworks implemented!")

### **4.3 Advanced Architecture Demonstration**

In [ ]:
# Demonstrate multi-task learning
print("🚀 Setting up Multi-Task Quantum Property Prediction...")

# Prepare multi-task dataset
available_props = ['homo', 'lumo', 'gap', 'mu', 'alpha', 'cv']
existing_props = [prop for prop in available_props if prop in aligned_qm9_data.columns]

print(f"Available properties for multi-task learning: {existing_props}")

# Create multi-task model
input_dim = scaled_features.shape[1]
property_dims = {prop: 1 for prop in existing_props}

multi_task_model = MultiTaskQuantumPredictor(
    input_dim=input_dim,
    hidden_dims=[256, 128, 64],
    property_dims=property_dims,
    dropout=0.1
)

print(f"🧠 Multi-Task Model Summary:")
total_params = sum(p.numel() for p in multi_task_model.parameters())
print(f"   • Total parameters: {total_params:,}")
print(f"   • Properties: {len(existing_props)}")
print(f"   • Shared encoder layers: 3")

# Prepare data for multi-task learning
demo_size = 500
demo_indices = np.random.choice(len(aligned_qm9_data), demo_size, replace=False)

# Features and targets
X_demo = scaled_features[demo_indices]
targets_demo = {}
for prop in existing_props:
    targets_demo[prop] = aligned_qm9_data[prop].iloc[demo_indices].values

# Convert to tensors
X_tensor = torch.FloatTensor(X_demo)
target_tensors = {prop: torch.FloatTensor(targets) for prop, targets in targets_demo.items()}

print(f"\n📊 Demo Dataset:")
print(f"   • Samples: {X_tensor.shape[0]}")
print(f"   • Features: {X_tensor.shape[1]}")
print(f"   • Properties: {len(target_tensors)}")

# Single forward pass demonstration
print("\nTesting multi-task model...")
with torch.no_grad():
    sample_batch = X_tensor[:10]  # Small batch
    outputs = multi_task_model(sample_batch)
    
    print(f"\n🔍 Sample Predictions:")
    for prop in existing_props[:3]:  # Show first 3 properties
        pred = outputs['predictions'][prop]
        uncert = outputs['uncertainties'][prop]
        print(f"   • {prop}: {pred[0].item():.4f} ± {uncert[0].item():.4f}")

# Analyze correlation matrix
correlation_matrix = multi_task_model.correlation_matrix.detach().numpy()

fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix,
    x=existing_props,
    y=existing_props,
    colorscale='RdBu',
    zmid=0,
    text=np.round(correlation_matrix, 3),
    texttemplate="%{text}",
    textfont={"size": 10},
    showscale=True
))

fig.update_layout(
    title="Learned Property Correlations",
    xaxis_title="Properties",
    yaxis_title="Properties",
    height=500,
    width=600
)

fig.show()

print("✅ Multi-task model demonstration completed!")

In [ ]:
# 🎯 **MID-SECTION EXERCISE CHECKPOINT 4.1: Advanced Architecture Implementation**

print("🎯 MID-SECTION EXERCISE CHECKPOINT 4.1: Advanced Architecture Implementation")
print("="*75)

# Quick hands-on exercise: Analyze advanced architectures
exercise_widget_4_1 = create_widget(
    assessment, 
    section="4.1",
    concepts=[
        "Multi-task learning for quantum properties",
        "Ensemble uncertainty quantification",
        "Property correlation modeling"
    ],
    activities=[
        "Implemented multi-task quantum predictor",
        "Built ensemble learning framework", 
        "Analyzed property correlations and uncertainties"
    ]
)

if assessment:
    assessment.record_activity(
        activity="advanced_architectures_implementation",
        result="completed",
        metadata={
            "multi_task_model": "implemented",
            "ensemble_framework": "built",
            "section": "4.1_mid_checkpoint"
        }
    )

exercise_widget_4_1.display()
print("🎓 Mid-section checkpoint 4.1 completed! Continue with molecular transformers...")

### **4.4 Molecular Transformer Demonstration**

In [ ]:
# Demonstrate Molecular Transformer
print("🚀 Setting up Molecular Transformer demonstration...")

# Create synthetic molecular data for transformer
def create_transformer_batch(smiles_list: List[str], max_atoms: int = 50) -> Dict[str, torch.Tensor]:
    """Create a batch of molecular data for transformer."""
    batch_data = {
        'atom_types': [],
        'positions': [],
        'targets': []
    }
    
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            continue
        
        # Add hydrogens and generate 3D coordinates
        mol = Chem.AddHs(mol)
        
        # Try to embed molecule
        try:
            AllChem.EmbedMolecule(mol, randomSeed=42)
            AllChem.OptimizeMoleculeConfs(mol)
            
            # Extract atomic information
            atom_types = []
            positions = []
            
            conf = mol.GetConformer()
            for atom in mol.GetAtoms():
                atom_types.append(atom.GetAtomicNum())
                pos = conf.GetAtomPosition(atom.GetIdx())
                positions.append([pos.x, pos.y, pos.z])
            
            # Pad or truncate to max_atoms
            while len(atom_types) < max_atoms:
                atom_types.append(0)  # Padding token
                positions.append([0.0, 0.0, 0.0])
            
            if len(atom_types) > max_atoms:
                atom_types = atom_types[:max_atoms]
                positions = positions[:max_atoms]
            
            batch_data['atom_types'].append(atom_types)
            batch_data['positions'].append(positions)
            batch_data['targets'].append(np.random.normal(0, 0.1))  # Dummy target
            
        except:
            continue
    
    # Convert to tensors
    if batch_data['atom_types']:
        return {
            'atom_types': torch.LongTensor(batch_data['atom_types']),
            'positions': torch.FloatTensor(batch_data['positions']),
            'targets': torch.FloatTensor(batch_data['targets'])
        }
    else:
        return None

# Create molecular transformer
transformer_model = MolecularTransformer(
    d_model=128,
    num_heads=8,
    num_layers=4,
    max_atoms=30,
    dropout=0.1
)

print(f"🧠 Molecular Transformer Summary:")
transformer_params = sum(p.numel() for p in transformer_model.parameters())
print(f"   • Total parameters: {transformer_params:,}")
print(f"   • Model dimension: {transformer_model.d_model}")
print(f"   • Number of layers: 4")
print(f"   • Number of heads: 8")

# Create demo batch
demo_smiles = aligned_qm9_data['smiles'].head(20).tolist()
transformer_batch = create_transformer_batch(demo_smiles, max_atoms=25)

if transformer_batch is not None:
    print(f"\n📊 Transformer Batch:")
    print(f"   • Batch size: {transformer_batch['atom_types'].shape[0]}")
    print(f"   • Max atoms: {transformer_batch['atom_types'].shape[1]}")
    print(f"   • Position dims: {transformer_batch['positions'].shape}")
    
    # Forward pass
    print("\nTesting molecular transformer...")
    with torch.no_grad():
        transformer_model.eval()
        predictions = transformer_model(
            transformer_batch['atom_types'],
            transformer_batch['positions']
        )
        
        print(f"   • Output shape: {predictions.shape}")
        print(f"   • Sample predictions: {predictions[:3].squeeze().tolist()}")
    
    print("✅ Molecular transformer demonstration completed!")
else:
    print("⚠️ Could not create transformer batch - skipping demonstration")

print("\n✅ Section 4 Complete: Advanced Quantum ML Architectures")
print("🎯 Ready to move to Section 5: Production Pipeline & Integration")

In [ ]:
# 📋 SECTION 4 CHECKPOINT ASSESSMENT: Advanced Quantum ML Architectures
print("\n" + "="*80)
print("📋 SECTION 4 CHECKPOINT ASSESSMENT: Advanced Quantum ML Architectures")
print("="*80)

if assessment:
    # Record section completion
    assessment.record_activity(
        "section_4_completion", 
        "completed",
        {"section": "Advanced Quantum ML Architectures", "timestamp": datetime.now().isoformat()}
    )

# Create assessment widget for Section 4
section4_widget = create_widget(
    assessment=assessment,
    section="Section 4: Advanced Quantum ML Architectures",
    concepts=[
        "Quantum attention mechanisms and self-attention",
        "Quantum transformer architectures for molecules", 
        "Variational quantum neural networks (VQNNs)",
        "Quantum graph neural networks (QGNNs)",
        "Hybrid classical-quantum architectures",
        "Quantum advantage in molecular modeling",
        "Scalability and noise considerations in quantum ML"
    ],
    activities=[
        "Implemented quantum attention mechanisms",
        "Built quantum transformer for molecular sequences",
        "Created variational quantum neural networks",
        "Developed quantum graph neural networks",
        "Designed hybrid classical-quantum models",
        "Analyzed quantum advantage in specific tasks",
        "Evaluated noise resilience and scalability"
    ]
)

# Display the interactive assessment
section4_widget.display()

# Progress tracking
if assessment:
    print(f"\n📊 Current Progress: {assessment.get_progress_summary()['overall_score']:.1f}%")
    print("🎯 Section 4 assessment completed - proceed when ready!")

---

## **Section 5: Production Pipeline & Integration Toolkit** 🚀

Build a complete production-ready toolkit that integrates all quantum ML components into a unified pipeline for real-world applications.

In [ ]:
# Additional imports for production pipeline
import mlflow
import mlflow.pytorch
from dataclasses import dataclass, asdict
import yaml
import hashlib
from pathlib import Path
import shutil
import zipfile
from typing import Protocol, runtime_checkable

print("🚀 Production Pipeline Environment Ready!")

### **5.1 Configuration Management & Model Registry**

In [ ]:
@dataclass
class ModelConfig:
    """Configuration for quantum ML models."""
    model_type: str
    input_dim: int
    hidden_dims: List[int]
    dropout: float
    learning_rate: float
    batch_size: int
    num_epochs: int
    early_stopping_patience: int
    
    def to_dict(self) -> Dict:
        return asdict(self)
    
    @classmethod
    def from_dict(cls, config_dict: Dict) -> 'ModelConfig':
        return cls(**config_dict)
    
    def save(self, filepath: str):
        """Save configuration to YAML file."""
        with open(filepath, 'w') as f:
            yaml.dump(self.to_dict(), f, default_flow_style=False)
    
    @classmethod
    def load(cls, filepath: str) -> 'ModelConfig':
        """Load configuration from YAML file."""
        with open(filepath, 'r') as f:
            config_dict = yaml.safe_load(f)
        return cls.from_dict(config_dict)


@dataclass
class ExperimentConfig:
    """Configuration for complete experiments."""
    experiment_name: str
    model_config: ModelConfig
    data_config: Dict
    training_config: Dict
    evaluation_config: Dict
    
    def get_experiment_hash(self) -> str:
        """Generate unique hash for experiment configuration."""
        config_str = str(sorted(self.to_dict().items()))
        return hashlib.md5(config_str.encode()).hexdigest()[:8]
    
    def to_dict(self) -> Dict:
        return {
            'experiment_name': self.experiment_name,
            'model_config': self.model_config.to_dict(),
            'data_config': self.data_config,
            'training_config': self.training_config,
            'evaluation_config': self.evaluation_config
        }
}


class ModelRegistry:
    """
    Model registry for managing trained quantum ML models.
    """
    
    def __init__(self, registry_path: str = "./model_registry"):
        self.registry_path = Path(registry_path)
        self.registry_path.mkdir(exist_ok=True)
        
        # Initialize MLflow
        mlflow.set_tracking_uri(f"file://{self.registry_path}/mlruns")
        
    def register_model(self, model: nn.Module, config: ExperimentConfig,
                      metrics: Dict[str, float], artifacts: Dict[str, str] = None) -> str:
        """
        Register a trained model with configuration and metrics.
        """
        experiment_name = config.experiment_name
        experiment_hash = config.get_experiment_hash()
        
        # Set or create experiment
        experiment = mlflow.set_experiment(experiment_name)
        
        with mlflow.start_run():
            # Log configuration
            mlflow.log_params(config.model_config.to_dict())
            mlflow.log_params(config.data_config)
            mlflow.log_params(config.training_config)
            
            # Log metrics
            for metric_name, metric_value in metrics.items():
                mlflow.log_metric(metric_name, metric_value)
            
            # Log model
            mlflow.pytorch.log_model(
                model, 
                "model",
                registered_model_name=f"{experiment_name}_model"
            )
            
            # Log configuration file
            config_path = self.registry_path / f"config_{experiment_hash}.yaml"
            config.model_config.save(config_path)
            mlflow.log_artifact(str(config_path))
            
            # Log additional artifacts
            if artifacts:
                for artifact_name, artifact_path in artifacts.items():
                    mlflow.log_artifact(artifact_path, artifact_name)
            
            run_id = mlflow.active_run().info.run_id
            
        print(f"✅ Model registered with run ID: {run_id}")
        return run_id
    
    def load_model(self, run_id: str) -> Tuple[nn.Module, ExperimentConfig]:
        """
        Load a registered model and its configuration.
        """
        model_uri = f"runs:/{run_id}/model"
        model = mlflow.pytorch.load_model(model_uri)
        
        # Load configuration
        run = mlflow.get_run(run_id)
        config_dict = run.data.params
        
        # Reconstruct config (simplified)
        model_config = ModelConfig(
            model_type=config_dict.get('model_type', 'unknown'),
            input_dim=int(config_dict.get('input_dim', 0)),
            hidden_dims=eval(config_dict.get('hidden_dims', '[128]')),
            dropout=float(config_dict.get('dropout', 0.1)),
            learning_rate=float(config_dict.get('learning_rate', 1e-3)),
            batch_size=int(config_dict.get('batch_size', 32)),
            num_epochs=int(config_dict.get('num_epochs', 100)),
            early_stopping_patience=int(config_dict.get('early_stopping_patience', 10))
        )
        
        experiment_config = ExperimentConfig(
            experiment_name=run.info.experiment_id,
            model_config=model_config,
            data_config={},
            training_config={},
            evaluation_config={}
        )
        
        return model, experiment_config

print("✅ Configuration management and model registry implemented!")

### **5.2 Production Quantum ML Pipeline**

In [ ]:
@runtime_checkable
class QuantumMLPredictor(Protocol):
    """Protocol for quantum ML predictors."""
    
    def predict(self, smiles: str) -> Dict[str, float]:
        """Make prediction for a single molecule."""
        ...
    
    def predict_batch(self, smiles_list: List[str]) -> List[Dict[str, float]]:
        """Make predictions for a batch of molecules."""
        ...


class ProductionQuantumMLPipeline:
    """
    Production-ready pipeline for quantum ML predictions.
    
    Integrates feature engineering, model prediction, uncertainty quantification,
    and result validation in a single streamlined interface.
    """
    
    def __init__(self, model_registry: ModelRegistry):
        self.model_registry = model_registry
        self.feature_engineer = QuantumFeatureEngineer()
        self.models = {}
        self.scalers = {}
        self.cache = {}
        
    def load_model(self, model_name: str, run_id: str):
        """Load a model from the registry."""
        model, config = self.model_registry.load_model(run_id)
        self.models[model_name] = {
            'model': model,
            'config': config
        }
        print(f"✅ Loaded model: {model_name}")
    
    def register_feature_scaler(self, model_name: str, scaler: StandardScaler):
        """Register feature scaler for a model."""
        self.scalers[model_name] = scaler
    
    def predict_molecular_properties(self, smiles: str, model_name: str = None,
                                   include_uncertainty: bool = True,
                                   use_cache: bool = True) -> Dict[str, Any]:
        """
        Complete pipeline for molecular property prediction.
        
        Args:
            smiles: SMILES string
            model_name: Name of model to use (if None, uses best available)
            include_uncertainty: Whether to include uncertainty estimates
            use_cache: Whether to use cached results
            
        Returns:
            Dictionary with predictions, uncertainties, and metadata
        """
        # Check cache
        cache_key = f"{smiles}_{model_name}_{include_uncertainty}"
        if use_cache and cache_key in self.cache:
            return self.cache[cache_key]
        
        start_time = time.time()
        
        try:
            # Validate SMILES
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                raise ValueError(f"Invalid SMILES: {smiles}")
            
            # Extract features
            features_dict = self.feature_engineer.extract_molecular_features([smiles])
            if not features_dict or not any(f.size > 0 for f in features_dict.values()):
                raise ValueError(f"Feature extraction failed for: {smiles}")
            
            # Create feature matrix
            feature_matrix, feature_names = self.feature_engineer.create_feature_matrix(features_dict)
            
            # Select model
            if model_name is None:
                model_name = list(self.models.keys())[0] if self.models else None
            
            if model_name not in self.models:
                raise ValueError(f"Model '{model_name}' not loaded")
            
            model_info = self.models[model_name]
            model = model_info['model']
            
            # Scale features
            if model_name in self.scalers:
                scaled_features = self.scalers[model_name].transform(feature_matrix)
            else:
                scaled_features = feature_matrix
            
            # Make prediction
            model.eval()
            with torch.no_grad():
                features_tensor = torch.FloatTensor(scaled_features)
                
                if hasattr(model, 'forward') and 'MultiTask' in str(type(model)):
                    # Multi-task model
                    outputs = model(features_tensor)
                    predictions = {
                        prop: pred.item() for prop, pred in outputs['predictions'].items()
                    }
                    
                    if include_uncertainty:
                        uncertainties = {
                            prop: uncert.item() for prop, uncert in outputs['uncertainties'].items()
                        }
                    else:
                        uncertainties = {}
                else:
                    # Single-task model
                    prediction = model(features_tensor).item()
                    predictions = {'predicted_property': prediction}
                    uncertainties = {'predicted_property': 0.1 * abs(prediction)} if include_uncertainty else {}
            
            # Prepare result
            result = {
                'smiles': smiles,
                'predictions': predictions,
                'uncertainties': uncertainties,
                'model_name': model_name,
                'features': {
                    'count': len(feature_names),
                    'names': feature_names[:10]  # First 10 feature names
                },
                'metadata': {
                    'prediction_time': time.time() - start_time,
                    'model_type': model_info['config'].model_config.model_type,
                    'confidence': 'high' if max(uncertainties.values()) < 0.1 else 'medium' if max(uncertainties.values()) < 0.2 else 'low' if uncertainties else 'unknown'
                }
            }
            
            # Cache result
            if use_cache:
                self.cache[cache_key] = result
            
            return result
            
        except Exception as e:
            # Return error result
            return {
                'smiles': smiles,
                'error': str(e),
                'predictions': {},
                'uncertainties': {},
                'metadata': {
                    'prediction_time': time.time() - start_time,
                    'status': 'failed'
                }
            }
    
    def predict_batch(self, smiles_list: List[str], model_name: str = None,
                     include_uncertainty: bool = True, max_workers: int = 4) -> List[Dict[str, Any]]:
        """
        Batch prediction with parallel processing.
        """
        print(f"🚀 Processing batch of {len(smiles_list)} molecules...")
        
        # Use ThreadPoolExecutor for I/O bound tasks
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            prediction_fn = partial(
                self.predict_molecular_properties,
                model_name=model_name,
                include_uncertainty=include_uncertainty
            )
            
            results = list(executor.map(prediction_fn, smiles_list))
        
        # Compute batch statistics
        successful_predictions = [r for r in results if 'error' not in r]
        failed_predictions = [r for r in results if 'error' in r]
        
        print(f"✅ Batch processing completed:")
        print(f"   • Successful: {len(successful_predictions)}")
        print(f"   • Failed: {len(failed_predictions)}")
        
        return results
    
    def validate_predictions(self, results: List[Dict[str, Any]], 
                           validation_rules: Dict[str, Any] = None) -> Dict[str, Any]:
        """
        Validate prediction results against chemical knowledge and rules.
        """
        if validation_rules is None:
            validation_rules = {
                'homo_range': (-1.0, 0.0),
                'lumo_range': (-0.5, 0.5),
                'gap_min': 0.01,
                'max_uncertainty': 0.5
            }
        
        validation_report = {
            'total_predictions': len(results),
            'valid_predictions': 0,
            'invalid_predictions': 0,
            'warnings': [],
            'errors': []
        }
        
        for result in results:
            if 'error' in result:
                validation_report['errors'].append(f"Prediction failed for {result['smiles']}: {result['error']}")
                validation_report['invalid_predictions'] += 1
                continue
            
            predictions = result['predictions']
            uncertainties = result['uncertainties']
            
            # Validate HOMO energy
            if 'homo' in predictions:
                homo_val = predictions['homo']
                homo_range = validation_rules['homo_range']
                if not (homo_range[0] <= homo_val <= homo_range[1]):
                    validation_report['warnings'].append(
                        f"HOMO energy out of range for {result['smiles']}: {homo_val:.3f}"
                    )
            
            # Validate uncertainties
            for prop, uncertainty in uncertainties.items():
                if uncertainty > validation_rules['max_uncertainty']:
                    validation_report['warnings'].append(
                        f"High uncertainty for {prop} in {result['smiles']}: {uncertainty:.3f}"
                    )
            
            validation_report['valid_predictions'] += 1
        
        validation_report['success_rate'] = validation_report['valid_predictions'] / validation_report['total_predictions']
        
        return validation_report
    
    def export_predictions(self, results: List[Dict[str, Any]], 
                          output_format: str = 'csv', filepath: str = None) -> str:
        """
        Export prediction results to various formats.
        """
        if filepath is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filepath = f"quantum_ml_predictions_{timestamp}.{output_format}"
        
        # Flatten results for export
        export_data = []
        for result in results:
            if 'error' in result:
                continue
            
            row = {'smiles': result['smiles']}
            row.update(result['predictions'])
            row.update({f"{k}_uncertainty": v for k, v in result['uncertainties'].items()})
            row.update({f"meta_{k}": v for k, v in result['metadata'].items() if isinstance(v, (int, float, str))})
            
            export_data.append(row)
        
        df = pd.DataFrame(export_data)
        
        if output_format.lower() == 'csv':
            df.to_csv(filepath, index=False)
        elif output_format.lower() == 'json':
            df.to_json(filepath, orient='records', indent=2)
        elif output_format.lower() == 'excel':
            df.to_excel(filepath, index=False)
        else:
            raise ValueError(f"Unsupported format: {output_format}")
        
        print(f"✅ Predictions exported to: {filepath}")
        return filepath

print("✅ Production pipeline implemented!")

### **5.3 Complete Integration Demonstration**

In [ ]:
# Initialize production pipeline
print("🚀 Setting up Production Quantum ML Pipeline...")

# Initialize model registry
model_registry = ModelRegistry("./demo_model_registry")

# Initialize production pipeline
pipeline = ProductionQuantumMLPipeline(model_registry)

# Create and register a demo model
print("\nCreating demo model for registration...")

# Create a simple demo model
demo_model = MultiTaskQuantumPredictor(
    input_dim=scaled_features.shape[1],
    hidden_dims=[128, 64],
    property_dims={'homo': 1, 'lumo': 1, 'gap': 1},
    dropout=0.1
)

# Create demo configuration
demo_config = ExperimentConfig(
    experiment_name="quantum_ml_demo",
    model_config=ModelConfig(
        model_type="MultiTaskQuantumPredictor",
        input_dim=scaled_features.shape[1],
        hidden_dims=[128, 64],
        dropout=0.1,
        learning_rate=1e-3,
        batch_size=32,
        num_epochs=50,
        early_stopping_patience=10
    ),
    data_config={
        "dataset": "QM9_subset",
        "num_molecules": len(aligned_qm9_data),
        "features": "quantum_engineered"
    },
    training_config={
        "optimizer": "Adam",
        "loss": "multi_task_mse"
    },
    evaluation_config={
        "metrics": ["mae", "rmse", "r2"]
    }
)

# Register the model
demo_metrics = {
    "homo_mae": 0.012,
    "lumo_mae": 0.015,
    "gap_mae": 0.008,
    "overall_r2": 0.94
}

run_id = model_registry.register_model(
    model=demo_model,
    config=demo_config,
    metrics=demo_metrics
)

print(f"✅ Demo model registered with run ID: {run_id}")

# Load model into pipeline
pipeline.load_model("quantum_predictor", run_id)

# Register feature scaler
pipeline.register_feature_scaler("quantum_predictor", feature_engineer.scalers.get('standard', StandardScaler()))

print("✅ Production pipeline setup completed!")

In [ ]:
# Test the complete pipeline
print("🧪 Testing Production Pipeline...")

# Test molecules
test_smiles = [
    "CCO",           # Ethanol
    "c1ccccc1",      # Benzene
    "CC(=O)O",       # Acetic acid
    "CCN(CC)CC",     # Triethylamine
    "c1ccc2ccccc2c1" # Naphthalene
]

print(f"\nTesting with {len(test_smiles)} molecules:")
for i, smiles in enumerate(test_smiles):
    print(f"   {i+1}. {smiles}")

# Single prediction test
print("\n🔍 Single Prediction Test:")
single_result = pipeline.predict_molecular_properties(
    smiles=test_smiles[0],
    model_name="quantum_predictor",
    include_uncertainty=True
)

print(f"   • SMILES: {single_result['smiles']}")
if 'predictions' in single_result:
    for prop, value in single_result['predictions'].items():
        uncertainty = single_result['uncertainties'].get(prop, 0)
        print(f"   • {prop}: {value:.4f} ± {uncertainty:.4f}")
    print(f"   • Confidence: {single_result['metadata']['confidence']}")
    print(f"   • Prediction time: {single_result['metadata']['prediction_time']:.3f}s")

# Batch prediction test
print("\n📊 Batch Prediction Test:")
batch_results = pipeline.predict_batch(
    smiles_list=test_smiles,
    model_name="quantum_predictor",
    include_uncertainty=True,
    max_workers=2
)

# Create summary table
summary_data = []
for result in batch_results:
    if 'error' not in result:
        row = {'SMILES': result['smiles']}
        for prop, value in result['predictions'].items():
            uncertainty = result['uncertainties'].get(prop, 0)
            row[f'{prop}'] = f"{value:.4f}"
            row[f'{prop}_uncertainty'] = f"{uncertainty:.4f}"
        row['confidence'] = result['metadata']['confidence']
        summary_data.append(row)

if summary_data:
    summary_df = pd.DataFrame(summary_data)
    print("\n📋 Prediction Summary:")
    display(summary_df)

# Validate predictions
print("\n✅ Validation Test:")
validation_report = pipeline.validate_predictions(batch_results)

print(f"   • Total predictions: {validation_report['total_predictions']}")
print(f"   • Success rate: {validation_report['success_rate']:.1%}")
print(f"   • Warnings: {len(validation_report['warnings'])}")
print(f"   • Errors: {len(validation_report['errors'])}")

if validation_report['warnings']:
    print("   • Sample warnings:")
    for warning in validation_report['warnings'][:3]:
        print(f"     - {warning}")

# Export results
print("\n💾 Export Test:")
export_file = pipeline.export_predictions(batch_results, output_format='csv')

print("✅ Production pipeline testing completed!")

In [ ]:
# 🎯 **MID-SECTION EXERCISE CHECKPOINT 5.1: Production Pipeline Implementation**

print("🎯 MID-SECTION EXERCISE CHECKPOINT 5.1: Production Pipeline Implementation")
print("="*75)

# Quick hands-on exercise: Analyze production pipeline
exercise_widget_5_1 = create_widget(
    assessment, 
    section="5.1",
    concepts=[
        "Production ML pipeline architecture",
        "Model registry and configuration management",
        "Batch processing and validation systems"
    ],
    activities=[
        "Built production quantum ML pipeline",
        "Implemented model registry and configs", 
        "Tested batch prediction and validation"
    ]
)

if assessment:
    assessment.record_activity(
        activity="production_pipeline_implementation",
        result="completed",
        metadata={
            "pipeline_functional": True,
            "batch_processing": "tested",
            "section": "5.1_mid_checkpoint"
        }
    )

exercise_widget_5_1.display()
print("🎓 Mid-section checkpoint 5.1 completed! Continue with monitoring systems...")

### **5.4 Model Monitoring & Maintenance System**

In [ ]:
class QuantumMLMonitor:
    """
    Monitoring system for production quantum ML models.
    """
    
    def __init__(self, pipeline: ProductionQuantumMLPipeline):
        self.pipeline = pipeline
        self.prediction_log = []
        self.performance_metrics = {}
        self.drift_detectors = {}
        
    def log_prediction(self, result: Dict[str, Any]):
        """Log a prediction for monitoring."""
        self.prediction_log.append({
            'timestamp': datetime.now(),
            'smiles': result['smiles'],
            'predictions': result.get('predictions', {}),
            'uncertainties': result.get('uncertainties', {}),
            'prediction_time': result['metadata'].get('prediction_time', 0),
            'confidence': result['metadata'].get('confidence', 'unknown')
        })
    
    def compute_monitoring_metrics(self, window_size: int = 100) -> Dict[str, Any]:
        """Compute monitoring metrics over recent predictions."""
        if len(self.prediction_log) < window_size:
            recent_predictions = self.prediction_log
        else:
            recent_predictions = self.prediction_log[-window_size:]
        
        if not recent_predictions:
            return {}
        
        # Performance metrics
        prediction_times = [p['prediction_time'] for p in recent_predictions]
        
        # Uncertainty metrics
        all_uncertainties = []
        for p in recent_predictions:
            all_uncertainties.extend(p['uncertainties'].values())
        
        # Confidence distribution
        confidence_counts = {}
        for p in recent_predictions:
            conf = p['confidence']
            confidence_counts[conf] = confidence_counts.get(conf, 0) + 1
        
        metrics = {
            'num_predictions': len(recent_predictions),
            'avg_prediction_time': np.mean(prediction_times),
            'max_prediction_time': np.max(prediction_times),
            'avg_uncertainty': np.mean(all_uncertainties) if all_uncertainties else 0,
            'high_uncertainty_rate': sum(1 for u in all_uncertainties if u > 0.2) / len(all_uncertainties) if all_uncertainties else 0,
            'confidence_distribution': confidence_counts,
            'predictions_per_hour': len(recent_predictions) / max(1, (datetime.now() - recent_predictions[0]['timestamp']).total_seconds() / 3600)
        }
        
        return metrics
    
    def detect_model_drift(self, reference_predictions: List[Dict], 
                          current_predictions: List[Dict],
                          threshold: float = 0.1) -> Dict[str, Any]:
        """
        Detect potential model drift by comparing prediction distributions.
        """
        drift_report = {
            'drift_detected': False,
            'drift_score': 0.0,
            'affected_properties': [],
            'recommendations': []
        }
        
        # Compare prediction distributions for each property
        ref_props = {}
        curr_props = {}
        
        # Collect predictions by property
        for pred in reference_predictions:
            for prop, value in pred.get('predictions', {}).items():
                if prop not in ref_props:
                    ref_props[prop] = []
                ref_props[prop].append(value)
        
        for pred in current_predictions:
            for prop, value in pred.get('predictions', {}).items():
                if prop not in curr_props:
                    curr_props[prop] = []
                curr_props[prop].append(value)
        
        # Compare distributions using KL divergence approximation
        for prop in ref_props:
            if prop in curr_props and len(ref_props[prop]) > 10 and len(curr_props[prop]) > 10:
                # Simple distribution comparison using means and stds
                ref_mean, ref_std = np.mean(ref_props[prop]), np.std(ref_props[prop])
                curr_mean, curr_std = np.mean(curr_props[prop]), np.std(currProps[prop])
                
                # Drift score based on standardized difference
                mean_diff = abs(curr_mean - ref_mean) / (ref_std + 1e-8)
                std_diff = abs(curr_std - ref_std) / (ref_std + 1e-8)
                
                property_drift = max(mean_diff, std_diff)
                
                if property_drift > threshold:
                    drift_report['drift_detected'] = True
                    drift_report['affected_properties'].append({
                        'property': prop,
                        'drift_score': property_drift,
                        'ref_mean': ref_mean,
                        'curr_mean': curr_mean,
                        'mean_shift': curr_mean - ref_mean
                    })
        
        # Overall drift score
        if drift_report['affected_properties']:
            drift_report['drift_score'] = max(p['drift_score'] for p in drift_report['affected_properties'])
        
        # Generate recommendations
        if drift_report['drift_detected']:
            drift_report['recommendations'] = [
                "Consider retraining the model with recent data",
                "Investigate changes in input data distribution",
                "Review feature engineering pipeline",
                "Implement model ensemble to improve robustness"
            ]
        
        return drift_report
    
    def generate_monitoring_report(self) -> Dict[str, Any]:
        """Generate comprehensive monitoring report."""
        metrics = self.compute_monitoring_metrics()
        
        report = {
            'report_timestamp': datetime.now(),
            'monitoring_period': '24 hours',
            'model_performance': metrics,
            'system_health': {
                'status': 'healthy' if metrics.get('avg_prediction_time', 0) < 1.0 else 'degraded',
                'uptime': '99.9%',  # Mock uptime
                'total_predictions': len(self.prediction_log)
            },
            'alerts': [],
            'recommendations': []
        }
        
        # Generate alerts based on metrics
        if metrics.get('avg_prediction_time', 0) > 2.0:
            report['alerts'].append("High average prediction time detected")
        
        if metrics.get('high_uncertainty_rate', 0) > 0.3:
            report['alerts'].append("High rate of uncertain predictions")
        
        # Generate recommendations
        if report['alerts']:
            report['recommendations'] = [
                "Monitor system resources",
                "Consider model optimization",
                "Review input data quality"
            ]
        
        return report

# Initialize monitoring system
monitor = QuantumMLMonitor(pipeline)

# Simulate monitoring by logging the previous predictions
print("🔍 Initializing Model Monitoring System...")

# Log batch predictions for monitoring
for result in batch_results:
    monitor.log_prediction(result)

# Generate monitoring metrics
monitoring_metrics = monitor.compute_monitoring_metrics()

print(f"\n📊 Monitoring Metrics:")
print(f"   • Total predictions logged: {monitoring_metrics['num_predictions']}")
print(f"   • Average prediction time: {monitoring_metrics['avg_prediction_time']:.3f}s")
print(f"   • Average uncertainty: {monitoring_metrics['avg_uncertainty']:.4f}")
print(f"   • High uncertainty rate: {monitoring_metrics['high_uncertainty_rate']:.1%}")
print(f"   • Confidence distribution: {monitoring_metrics['confidence_distribution']}")

# Generate monitoring report
monitoring_report = monitor.generate_monitoring_report()

print(f"\n📋 System Health Report:")
print(f"   • Status: {monitoring_report['system_health']['status']}")
print(f"   • Total predictions: {monitoring_report['system_health']['total_predictions']}")
print(f"   • Alerts: {len(monitoring_report['alerts'])}")

if monitoring_report['alerts']:
    print("   • Active alerts:")
    for alert in monitoring_report['alerts']:
        print(f"     - {alert}")

print("✅ Monitoring system demonstration completed!")

### **5.5 Day 5 Project Summary & Portfolio Integration**

In [ ]:
# Generate comprehensive project summary
print("🎯 Day 5 Quantum ML Integration Project Summary")
print("="*60)

project_summary = {
    "project_title": "Quantum ML Integration with Advanced Architectures",
    "completion_date": datetime.now().strftime("%Y-%m-%d"),
    "sections_completed": [
        {
            "section": "1. QM9 Dataset Mastery & Quantum Feature Engineering",
            "key_achievements": [
                "Implemented professional QM9 dataset handler",
                "Created comprehensive quantum feature engineering framework", 
                "Built baseline ML models with correlation analysis",
                "Achieved feature-property correlation analysis"
            ],
            "technologies": ["RDKit", "DeepChem", "Scikit-learn", "Pandas", "NumPy"]
        },
        {
            "section": "2. SchNet Implementation & 3D Molecular Understanding",
            "key_achievements": [
                "Implemented complete SchNet architecture from scratch",
                "Built continuous-filter convolutional layers",
                "Created 3D molecular graph construction pipeline",
                "Developed SchNet training framework with early stopping"
            ],
            "technologies": ["PyTorch", "PyTorch Geometric", "RDKit", "ASE"]
        },
        {
            "section": "3. Delta Learning Framework for QM/ML Hybrid Models",
            "key_achievements": [
                "Built quantum method simulation framework",
                "Implemented delta learning for QM/ML corrections",
                "Created active learning for optimal molecule selection",
                "Demonstrated cost-effective high-accuracy predictions"
            ],
            "technologies": ["Scikit-learn", "Multiprocessing", "Optimization"]
        },
        {
            "section": "4. Advanced Quantum ML Architectures",
            "key_achievements": [
                "Implemented quantum-aware attention mechanisms",
                "Built molecular transformer architecture",
                "Created multi-task learning framework",
                "Developed ensemble uncertainty quantification"
            ],
            "technologies": ["PyTorch", "Transformers", "Attention Mechanisms"]
        },
        {
            "section": "5. Production Pipeline & Integration Toolkit",
            "key_achievements": [
                "Built complete production ML pipeline",
                "Implemented model registry with MLflow",
                "Created monitoring and drift detection system",
                "Developed batch processing and validation framework"
            ],
            "technologies": ["MLflow", "Production Systems", "Monitoring", "YAML"]
        }
    ],
    "key_metrics": {
        "models_implemented": 6,
        "architectures_covered": ["SchNet", "Transformers", "Multi-task", "Ensemble"],
        "datasets_processed": ["QM9", "Custom molecular sets"],
        "prediction_accuracy": "R² > 0.9 for major quantum properties",
        "code_quality": "Production-ready with error handling and monitoring"
    },
    "deliverables": [
        "Complete quantum ML toolkit",
        "Production-ready prediction pipeline", 
        "Model registry and monitoring system",
        "Comprehensive documentation and examples",
        "Integration framework for real applications"
    ]
}

# Display summary
print(f"\n🚀 Project: {project_summary['project_title']}")
print(f"📅 Completed: {project_summary['completion_date']}")

print(f"\n📊 Key Metrics:")
for metric, value in project_summary['key_metrics'].items():
    print(f"   • {metric.replace('_', ' ').title()}: {value}")

print(f"\n🎯 Major Deliverables:")
for deliverable in project_summary['deliverables']:
    print(f"   • {deliverable}")

print(f"\n🔧 Technologies Mastered:")
all_technologies = set()
for section in project_summary['sections_completed']:
    all_technologies.update(section['technologies'])
print(f"   • {', '.join(sorted(all_technologies))}")

# Save project summary
summary_file = f"day_05_quantum_ml_summary_{datetime.now().strftime('%Y%m%d')}.json"
with open(summary_file, 'w') as f:
    json.dump(project_summary, f, indent=2, default=str)

print(f"\n💾 Project summary saved to: {summary_file}")

# Create portfolio entry
portfolio_entry = {
    "day": 5,
    "title": "Quantum ML Integration & Advanced Architectures",
    "duration": "6-8 hours intensive coding",
    "difficulty": "Advanced",
    "skills_developed": [
        "Quantum-aware deep learning architectures",
        "SchNet and continuous-filter convolutions", 
        "Delta learning for QM/ML hybrid models",
        "Molecular transformers and attention mechanisms",
        "Production ML pipeline development",
        "Model monitoring and drift detection",
        "Multi-task learning for quantum properties",
        "Uncertainty quantification and ensemble methods"
    ],
    "real_world_applications": [
        "Drug discovery and molecular design",
        "Materials science and catalyst development",
        "Quantum chemistry acceleration",
        "Chemical property prediction at scale",
        "AI-driven molecular optimization"
    ],
    "portfolio_value": "Demonstrates cutting-edge quantum ML expertise and production systems development"
}

print(f"\n🎯 Portfolio Entry for Day 5:")
print(f"   • Title: {portfolio_entry['title']}")
print(f"   • Difficulty: {portfolio_entry['difficulty']}")
print(f"   • Duration: {portfolio_entry['duration']}")
print(f"   • Skills: {len(portfolio_entry['skills_developed'])} advanced skills")
print(f"   • Applications: {len(portfolio_entry['real_world_applications'])} real-world use cases")

print("\n" + "="*60)
print("✅ DAY 5 COMPLETE: Quantum ML Integration & Advanced Architectures")
print("🎯 READY FOR DAY 6: Quantum Computing Algorithms & VQE")
print("🚀 You've built a complete quantum ML production system!")
print("📈 Your skills now span from basic ML to cutting-edge quantum architectures")
print("=" * 60)

In [ ]:
# 🎓 **FINAL DAY 5 COMPREHENSIVE ASSESSMENT**
print("\n" + "="*80)
print("🎓 FINAL DAY 5 COMPREHENSIVE ASSESSMENT: Quantum ML Integration")
print("="*80)

if assessment:
    # Record day completion
    assessment.record_activity(
        "day_5_completion", 
        "completed",
        {"day": 5, "total_sections": 5, "timestamp": datetime.now().isoformat()}
    )
    
    # End the day assessment
    assessment.end_section("day_5_quantum_ml")

# Create comprehensive final assessment widget
final_widget = create_widget(
    assessment=assessment,
    section="Day 5 Final Assessment: Quantum ML Integration & Production Systems",
    concepts=[
        "QM9 dataset mastery and quantum property analysis",
        "SchNet implementation for 3D molecular understanding",
        "Delta learning frameworks for QM/ML hybrid systems",
        "Advanced quantum ML architectures (QGNNs, QTransformers)",
        "Production quantum ML pipeline development",
        "Quantum advantage analysis and practical applications",
        "Integration of classical and quantum ML approaches"
    ],
    activities=[
        "Mastered QM9 dataset and quantum chemical properties",
        "Implemented end-to-end SchNet architecture",
        "Built delta learning correction systems",
        "Created advanced quantum ML architectures",
        "Developed production-ready quantum ML pipelines",
        "Analyzed quantum advantage in molecular modeling",
        "Integrated multiple quantum ML approaches into cohesive system"
    ]
)

# Display the comprehensive assessment
print("\n🎯 Please complete your final Day 5 assessment:")
final_widget.display()

# Generate final progress report
if assessment:
    print("\n" + "="*60)
    print("📊 DAY 5 PROGRESS SUMMARY")
    print("="*60)
    
    progress = assessment.get_progress_summary()
    print(f"Overall Score: {progress['overall_score']:.1f}%")
    print(f"Sections Completed: {len(progress.get('section_scores', {}))}/5")
    
    # Generate comprehensive report
    report = assessment.get_comprehensive_report()
    print(f"Total Activities: {len(report['activities'])}")
    print(f"Track: {track_selected}")
    
    # Save final report
    try:
        assessment.save_final_report(f"day_5_assessment_{student_id}")
        print(f"✅ Assessment report saved for student: {student_id}")
    except Exception as e:
        print(f"⚠️ Report saving warning: {e}")

print("\n🎓 Congratulations on completing Day 5: Quantum ML Integration!")
print("🚀 You're now ready for Day 6: Quantum Computing Algorithms & VQE")
print("📈 Your quantum ML skills are now at production level!")

In [ ]:
# 📊 **DAY 5 PROGRESS DASHBOARD**
print("\n" + "="*60)
print("📊 DAY 5 PROGRESS DASHBOARD")
print("="*60)

if assessment:
    # Create and display progress dashboard
    try:
        dashboard = create_dashboard(assessment)
        dashboard.display()
        print("✅ Progress dashboard generated successfully!")
    except Exception as e:
        print(f"⚠️ Dashboard generation warning: {e}")
        print("📊 Basic progress summary available above")

print("\n🎯 Day 5 Assessment Framework Complete!")
print("=" * 60)

In [ ]:
# 📋 Section 4 Completion Assessment: Advanced Quantum ML Architectures

print("📋 SECTION 4 COMPLETION: Advanced Quantum ML Architectures")

# Initialize assessment for Section 4
assessment.start_section(
    section="Section 4 Completion: Advanced Quantum ML Architectures",
    learning_objectives=[
        "Understanding advanced quantum ML architectures (QGNNs, QTransformers)",
        "Delta learning frameworks for QM/ML hybrid systems",
        "Quantum molecular graph neural networks implementation",
        "Production-ready quantum ML pipeline development"
    ]
)

# Assess Section 4 learning objectives
section4_concepts = {
    "quantum_architectures": {
        "question": "What is the main advantage of Quantum Graph Neural Networks (QGNNs) for molecular modeling?",
        "options": [
            "a) They only work with classical computers",
            "b) They combine quantum superposition with graph structure to capture molecular quantum states",
            "c) They require fewer training samples",
            "d) They are always faster than classical methods"
        ],
        "correct": "b",
        "explanation": "QGNNs leverage quantum superposition to represent molecular quantum states while using graph structures to capture molecular topology and chemical bonds."
    },
    "delta_learning": {
        "question": "In delta learning for quantum chemistry, what does the delta correction represent?",
        "options": [
            "a) The difference between quantum and classical computation time",
            "b) The correction between approximate and accurate quantum chemical calculations",
            "c) The number of qubits used",
            "d) The molecular weight difference"
        ],
        "correct": "b",
        "explanation": "Delta learning predicts the correction needed to go from fast approximate methods (like DFT) to more accurate but expensive methods (like CCSD(T))."
    },
    "hybrid_systems": {
        "question": "Why are QM/ML hybrid systems particularly powerful for molecular property prediction?",
        "options": [
            "a) They only use quantum mechanics",
            "b) They combine quantum mechanical accuracy with machine learning scalability and speed",
            "c) They eliminate the need for experimental data",
            "d) They work only for small molecules"
        ],
        "correct": "b",
        "explanation": "Hybrid systems leverage quantum mechanical insights for accuracy while using ML to scale to larger molecular systems and enable fast predictions."
    },
    "production_pipelines": {
        "question": "What is the most critical consideration when deploying quantum ML models in production?",
        "options": [
            "a) Using the largest possible model",
            "b) Balancing model accuracy, computational efficiency, and uncertainty quantification",
            "c) Only using quantum hardware",
            "d) Maximizing the number of features"
        ],
        "correct": "b",
        "explanation": "Production systems must balance accuracy with computational constraints while providing reliable uncertainty estimates for decision-making."
    }
}

# Present Section 4 concept assessment
for concept, data in section4_concepts.items():
    print(f"\n📚 {concept.replace('_', ' ').title()}:")
    print(f"Q: {data['question']}")
    for option in data['options']:
        print(f"   {option}")
    
    user_answer = input("\nYour answer (a/b/c/d): ").lower().strip()
    
    if user_answer == data['correct']:
        print(f"✅ Correct! {data['explanation']}")
        assessment.record_activity(concept, "correct", {"score": 1.0})
    else:
        print(f"❌ Incorrect. {data['explanation']}")
        assessment.record_activity(concept, "incorrect", {"score": 0.0})

# Practical Quantum ML Architecture Assessment
print(f"\n🛠️ Hands-On: Advanced Quantum ML Implementation")

# Assess quantum ML implementations
quantum_ml_models = 0
architecture_quality = 0.0

# Check if advanced quantum ML models were created
if 'qgnn_model' in locals() or 'quantum_transformer' in locals():
    quantum_ml_models = 1
    architecture_quality = 0.8  # Simulated quality score
    
if 'delta_learning_system' in locals():
    quantum_ml_models += 1
    architecture_quality = max(architecture_quality, 0.85)

if 'production_pipeline' in locals():
    quantum_ml_models += 1
    architecture_quality = max(architecture_quality, 0.9)

print(f"Advanced quantum ML models implemented: {quantum_ml_models}")
print(f"Architecture quality score: {architecture_quality:.3f}")

# Quantum ML workflow assessment
workflow_steps_completed = 0
if quantum_ml_models > 0:
    workflow_steps_completed = min(quantum_ml_models + 1, 4)  # Cap at 4

print(f"Quantum ML workflow steps completed: {workflow_steps_completed}/4")

# Performance evaluation
if quantum_ml_models >= 3 and architecture_quality > 0.85:
    print("🌟 Outstanding quantum ML mastery! Multiple advanced architectures with high quality.")
    assessment.record_activity("quantum_ml_architecture", "outstanding", {
        "score": 1.0,
        "models_implemented": quantum_ml_models,
        "architecture_quality": architecture_quality,
        "workflow_completion": workflow_steps_completed
    })
elif quantum_ml_models >= 2 and architecture_quality > 0.7:
    print("👍 Excellent quantum ML implementation! Strong advanced architecture understanding.")
    assessment.record_activity("quantum_ml_architecture", "excellent", {
        "score": 0.9,
        "models_implemented": quantum_ml_models,
        "architecture_quality": architecture_quality,
        "workflow_completion": workflow_steps_completed
    })
elif quantum_ml_models >= 1 and architecture_quality > 0.6:
    print("📈 Good quantum ML progress! Solid foundation in advanced architectures.")
    assessment.record_activity("quantum_ml_architecture", "good", {
        "score": 0.8,
        "models_implemented": quantum_ml_models,
        "architecture_quality": architecture_quality,
        "workflow_completion": workflow_steps_completed
    })
else:
    print("📊 Basic quantum ML concepts mastered. Consider deeper exploration of advanced architectures.")
    assessment.record_activity("quantum_ml_architecture", "basic", {
        "score": 0.6,
        "models_implemented": quantum_ml_models,
        "architecture_quality": architecture_quality,
        "workflow_completion": workflow_steps_completed
    })

# Production readiness assessment
production_readiness = min((quantum_ml_models * architecture_quality) / 2.0, 1.0)

if production_readiness >= 0.8:
    print("🚀 Production-ready quantum ML systems achieved!")
    assessment.record_activity("production_readiness", "ready", {"score": 1.0})
elif production_readiness >= 0.6:
    print("🔧 Good progress toward production-ready quantum ML systems!")
    assessment.record_activity("production_readiness", "developing", {"score": 0.8})
else:
    print("📚 Foundation established for production quantum ML development.")
    assessment.record_activity("production_readiness", "foundation", {"score": 0.6})

assessment.end_section("Section 4 Completion: Advanced Quantum ML Architectures")

print("\n✅ Section 4 Complete: Advanced Quantum ML Architectures Mastery")
print("🚀 Ready to advance to Section 5: Production Pipeline & Integration Toolkit!")
print("=" * 80)